In [2]:
#Imports
import numpy as np
import pandas as pd
import geopandas as gpd
import difflib
import matplotlib.pyplot as plt
import time

def  processEvents(dataFilepath_sim, nrows):
    PTs = []
    PEVs = []
    print('read', dataFilepath_sim)
    for chunk in pd.read_csv(dataFilepath_sim, compression = 'gzip',chunksize=1500000, nrows = nrows):
        if sum((chunk['type'] == 'PathTraversal')) > 0:
            chunk['vehicle'] = chunk['vehicle'].astype(str)
            PT = chunk.loc[(chunk['type'] == 'PathTraversal') & (chunk['length'] > 0)].dropna(how='all', axis=1)
            PT['departureTime'] = PT['departureTime'].astype(int)
            PT['arrivalTime'] = PT['arrivalTime'].astype(int)

            PTs.append(PT[['driver', 'vehicle', 'mode', 'length', 'startX', 'startY', 'endX', 'endY', 'vehicleType',
                           'arrivalTime', 'departureTime', 'primaryFuel', 'primaryFuelType', 'secondaryFuel',
                           'secondaryFuelType', 'numPassengers', 'riders','time']])
            print(chunk.type.value_counts())
            PEV = chunk.loc[(chunk.type == "PersonEntersVehicle") &
                            ~(chunk['person'].apply(str).str.contains('Agent').fillna(False)) &
                            ~(chunk['vehicle'].str.contains('body').fillna(False)), :].dropna(how='all', axis=1)
            if ~PEV.empty:
                PEV['time'] = PEV['time'].astype(int)
                PEVs.append(PEV)

    Events_file_sim_PT = pd.concat(PTs)[['vehicle','time','endY','endX','startY','startX','mode']]
    Events_file_sim_PE = pd.concat(PEVs)[['vehicle','time',]]
    print(Events_file_sim_PE)

    return Events_file_sim_PT, Events_file_sim_PE



#Filter transit trips
def filter_transit(Events_file_sim_PT,Events_file_sim_PE):
    print(Events_file_sim_PT['mode'].value_counts())
    Events_file_sim_PT = Events_file_sim_PT[(Events_file_sim_PT['mode'] == 'bus')|
                                            (Events_file_sim_PT['mode'] == 'tram')|
                                            (Events_file_sim_PT['mode'] == 'subway')|
                                            (Events_file_sim_PT['mode'] == 'cable_car')|
                                            (Events_file_sim_PT['mode'] == 'ferry')|
                                            (Events_file_sim_PT['mode'] == 'rail')]
    print(Events_file_sim_PT['mode'].value_counts())
    Events_file_sim_PE = Events_file_sim_PE[Events_file_sim_PE['vehicle'].isin(Events_file_sim_PT['vehicle'])]
    
    return Events_file_sim_PE

def guess_agency(Events_file_sim_PE):

    agencies = []
    for vehicleID in Events_file_sim_PE['vehicle']:
        agency = vehicleID.split(':')[0]
        if agency == 'petalumatransit-petaluma-ca-us':
            agencies.append('PE')
        elif agency == 'westcat-ca-us':
            agencies.append('WC')
        elif agency == 'caltrain-ca-us':
            agencies.append('CA')
        elif agency == 'riovista-ca-us':
            agencies.append('RV')
        elif agency == 'unioncity-ca-us':
            agencies.append('UC')
        else:
            if len(agency) == 2:
                agencies.append(agency)
            elif agency == 'Caltrain':
                agencies.append('CA')
            else:
                print('Warning, this agency is not recognized:', agency)
    Events_file_sim_PE['agency'] = agencies

    print(np.unique(agencies))
    
    return Events_file_sim_PE

def guess_route(Events_file_sim_PE, GTFS_filepaths):

    GTFS_trip_files = {}

    for GTFS_filepath, GTFS in zip(GTFS_filepaths,GTFSs):
        GTFS_trip_files[GTFS] = pd.read_csv(GTFS_filepath+'trips.txt')

    route_ids = []
    total_routes = len(Events_file_sim_PE['vehicle'])
    i = 0
    time_start = time.time()
    for vehicle, agency in zip(Events_file_sim_PE['vehicle'],Events_file_sim_PE['agency']):
        i+=1
        if i%10000 ==0:
            print(i,'/',total_routes,'. Time = ', time.time()-time_start, '. Estimated remaining time:', (time.time()-time_start)/i*total_routes-(time.time()-time_start))
        if agency == 'SM':
            route_ids.append(agency+':'+str(list(GTFS_trip_files[agency]['route_id'][
                GTFS_trip_files[agency]['trip_id'].astype(str)==
                     str(vehicle).split(':')[1]+'|'
                     +str(vehicle).split(':')[2]+':'
                     +str(vehicle).split(':')[3]+'|'
                     +str(vehicle).split(':')[4]+':'
                     +str(vehicle).split(':')[5]+':'
                     +str(vehicle).split(':')[6]])[0]))
        else:
#             print(list(GTFS_trip_files[agency]['route_id'][GTFS_trip_files[agency]['trip_id'].astype(str)==str(vehicle.split(':')[1])])[0])
            try:
                route_ids.append(agency+':'+str(list(GTFS_trip_files[agency]['route_id'][GTFS_trip_files[agency]['trip_id'].astype(str)==str(vehicle.split(':')[1])])[0]))
            except:
                print('Warning, trip non found for vehicle', vehicle)
                route_ids.append('tripID not found')
    #     elif agency == 'GG':
    #         route_ids.append('GG:'+str(list(GTFS_trip_files[agency]['route_id'][GTFS_trip_files[agency]['trip_id']==vehicle.split(':')[1]])[0]))
    #     elif agency == 'SF':
    #         route_ids.append('SF:'+str(list(GTFS_trip_files[agency]['route_id'][GTFS_trip_files[agency]['trip_id']==vehicle.split(':')[1]])[0]))
    Events_file_sim_PE['route_id'] = route_ids
    
    return Events_file_sim_PE



# def addGeometryIdToDataFrame(df, gdf, xcol, ycol, idColumn="geometry", df_geom='epsg:4326'):
#     gdf_data = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df[xcol], df[ycol]))
#     gdf_data.crs = {'init': df_geom}
#     joined = gpd.sjoin(gdf_data.to_crs('epsg:26910'), gdf.to_crs('epsg:26910'))
#     gdf_data = gdf_data.merge(joined['ZCTA'], left_index=True, right_index=True, how="left")
#     gdf_data.rename(columns={'ZCTA': idColumn}, inplace=True)
#     df = pd.DataFrame(gdf_data.drop(columns='geometry'))
#     #df.drop(columns=[xcol, ycol], inplace=True)
#     return df.loc[~df.index.duplicated(keep='first'), :]


def addGeometryIdToDataFrame(df, gdf, xcol, ycol, idColumn="geometry", df_geom='epsg:32610'): 
    gdf.set_crs(epsg = "3310", inplace = True)
    gdf_data = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df[xcol], df[ycol]))
    gdf_data.set_crs(epsg = "32610", inplace = True)
    joined = gpd.sjoin(gdf_data.to_crs('epsg:26910'), gdf.to_crs('epsg:26910'))
    gdf_data = gdf_data.merge(joined['ZCTA'], left_index=True, right_index=True, how="left")
    gdf_data.rename(columns={'ZCTA': idColumn}, inplace=True)
    df = pd.DataFrame(gdf_data.drop(columns='geometry'))
#     df.drop(columns=[xcol, ycol], inplace=True)
    return df.loc[~df.index.duplicated(keep='first'), :]





#####################################################################################
# dataFilepath_sim = [
#                 's3://beam-outputs/pilates-outputs/sfbay-baseline-20220801/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz',
#                 's3://beam-outputs/pilates-outputs/sfbay-AC-SF-CA-20220801/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz',
#                 's3://beam-outputs/pilates-outputs/sfbay-baseline-20220801/beam/year-2019-iteration-5/ITERS/it.0/0.events.csv.gz',
#                 's3://beam-outputs/pilates-outputs/sfbay-AC-SF-CA-20220801/beam/year-2019-iteration-5/ITERS/it.0/0.events.csv.gz',
#                 's3://beam-outputs/pilates-outputs/sfbay-baseline-20220801/beam/year-2020-iteration-5/ITERS/it.0/0.events.csv.gz',
#                 's3://beam-outputs/pilates-outputs/sfbay-AC-SF-CA-20220801/beam/year-2020-iteration-5/ITERS/it.0/0.events.csv.gz',
#                 's3://beam-outputs/pilates-outputs/sfbay-baseline-20220801/beam/year-2021-iteration-5/ITERS/it.0/0.events.csv.gz',
#                 's3://beam-outputs/pilates-outputs/sfbay-AC-SF-CA-20220801/beam/year-2021-iteration-5/ITERS/it.0/0.events.csv.gz',
#                 's3://beam-outputs/pilates-outputs/sfbay-baseline-20220816/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz',
#                 's3://beam-outputs/pilates-outputs/sfbay-TR-20220812/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz',
#                     's3://beam-outputs/pilates-outputs/sfbay-baseline-20220816/beam/year-2019-iteration-5/ITERS/it.0/0.events.csv.gz',
#                 's3://beam-outputs/pilates-outputs/sfbay-TR-20220812/beam/year-2019-iteration-5/ITERS/it.0/0.events.csv.gz',
#                     's3://beam-outputs/pilates-outputs/sfbay-baseline-20220816/beam/year-2020-iteration-5/ITERS/it.0/0.events.csv.gz',
#                 's3://beam-outputs/pilates-outputs/sfbay-TR-20220812/beam/year-2020-iteration-5/ITERS/it.0/0.events.csv.gz',
#                     's3://beam-outputs/pilates-outputs/sfbay-baseline-20220816/beam/year-2021-iteration-5/ITERS/it.0/0.events.csv.gz',
#                 's3://beam-outputs/pilates-outputs/sfbay-TR-20220812/beam/year-2021-iteration-5/ITERS/it.0/0.events.csv.gz',
#             ]

                    
# years = ['2018','2019','2020','2021']
#####################################################################################
dataFilepath_sim = [
#                 's3://beam-outputs/pilates-outputs/sfbay-baseline-20220801/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz',
#                 's3://beam-outputs/pilates-outputs/sfbay-AC-SF-CA-20220801/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz',
#                 's3://beam-outputs/pilates-outputs/sfbay-baseline-20220801/beam/year-2019-iteration-5/ITERS/it.0/0.events.csv.gz',
#                 's3://beam-outputs/pilates-outputs/sfbay-AC-SF-CA-20220801/beam/year-2019-iteration-5/ITERS/it.0/0.events.csv.gz',
#                 's3://beam-outputs/pilates-outputs/sfbay-baseline-20220801/beam/year-2020-iteration-5/ITERS/it.0/0.events.csv.gz',
#                 's3://beam-outputs/pilates-outputs/sfbay-AC-SF-CA-20220801/beam/year-2020-iteration-5/ITERS/it.0/0.events.csv.gz',
#                 's3://beam-outputs/pilates-outputs/sfbay-baseline-20220801/beam/year-2021-iteration-5/ITERS/it.0/0.events.csv.gz',
#                 's3://beam-outputs/pilates-outputs/sfbay-AC-SF-CA-20220801/beam/year-2021-iteration-5/ITERS/it.0/0.events.csv.gz',
#                 's3://beam-outputs/pilates-outputs/sfbay-baseline-20220822/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz',
#                 's3://beam-outputs/pilates-outputs/test_49_3/beam/year-2018-iteration-3/ITERS/it.0/0.events.csv.gz',
#                     's3://beam-outputs/pilates-outputs/sfbay-baseline-20220816/beam/year-2019-iteration-5/ITERS/it.0/0.events.csv.gz',
#                 's3://beam-outputs/pilates-outputs/sfbay-TR-20220812/beam/year-2019-iteration-5/ITERS/it.0/0.events.csv.gz',
#                     's3://beam-outputs/pilates-outputs/sfbay-baseline-20220816/beam/year-2020-iteration-5/ITERS/it.0/0.events.csv.gz',
#                 's3://beam-outputs/pilates-outputs/sfbay-TR-20220812/beam/year-2020-iteration-5/ITERS/it.0/0.events.csv.gz',
#                     's3://beam-outputs/pilates-outputs/sfbay-baseline-20220816/beam/year-2021-iteration-5/ITERS/it.0/0.events.csv.gz',
#                 's3://beam-outputs/pilates-outputs/sfbay-TR-20220812/beam/year-2021-iteration-5/ITERS/it.0/0.events.csv.gz',
                's3://beam-outputs/pilates-outputs/sfbay_baseline_20230224/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz',
                's3://beam-outputs/pilates-outputs/sfbay_baseline_TR_20230224/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz',
#                 's3://beam-outputs/pilates-outputs/sfbay-baseline2_0-20230222/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz',
#                 's3://beam-outputs/pilates-outputs/sfbay-baseline-TR-20230222/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz',
#                 's3://beam-outputs/pilates-outputs/sfbay-baseline2_0-20230222/beam/year-2018-iteration-10/ITERS/it.0/0.events.csv.gz',
#                 's3://beam-outputs/pilates-outputs/sfbay-baseline-TR-20230222/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz',
#                 's3://beam-outputs/pilates-outputs/sfbay-baseline2_0-20230222/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz',
#                 's3://beam-outputs/pilates-outputs/sfbay-baseline-TR-20230222/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz',
#                 's3://beam-outputs/pilates-outputs/sfbay-baseline2_0-20230222/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz',
#                 's3://beam-outputs/pilates-outputs/sfbay-baseline-TR-20230222/beam/year-2018-iteration-5/ITERS/it.0/0.events.csv.gz',
            ]

                    
years = ['2019']

names = []
for year in years:
    names.append('Baseline'+year)
    names.append('Future'+year)

output_filepath = '/Users/cpoliziani/Downloads/Transit Rich/Results/'

GTFS_filepath = '/Users/cpoliziani/Documents/repo/beam-data-sfbay/r5-simple-no-local/'
GTFS_filepath2 = '/Users/cpoliziani/Documents/repo/beam-data-sfbay/r5-simple-no-local-TR/'

RA_output = 'agency_ridershipNewit5.csv'
RR_output = 'route_ridershipNewit5.csv'

GTFSs = [
#     '3D',
         'AC','AM','AY','BA','CA','CC','CE','CM',
#     'CT',
    'DE','EM','FF','GG','HF','MA','PE','RV', 'SB',
        'SC','SF','SL','SM','SO','SR','ST','TD','UC','VC','VN','VT','WC','WH']

# GTFS_3D = GTFS_filepath+'3D/'
GTFS_AC = GTFS_filepath+'AC/'
GTFS_AM = GTFS_filepath+'AM/'
GTFS_AY = GTFS_filepath+'AY/'
GTFS_BA = GTFS_filepath+'BA/'
GTFS_Caltrain = GTFS_filepath+'Caltrain/'
GTFS_CC = GTFS_filepath+'CC/'
GTFS_CE = GTFS_filepath+'CE/'
GTFS_CM = GTFS_filepath+'CM/'
# GTFS_CT = GTFS_filepath+'CT/'
GTFS_DE = GTFS_filepath+'DE/'
GTFS_EM = GTFS_filepath+'EM/'
GTFS_FF = GTFS_filepath+'FF/'
GTFS_GG = GTFS_filepath+'GG/'
GTFS_HF = GTFS_filepath+'HF/'
GTFS_MA = GTFS_filepath+'MA/'
GTFS_PE = GTFS_filepath+'PE/'
GTFS_RV = GTFS_filepath+'RV/'
GTFS_SB = GTFS_filepath+'SB/'
GTFS_SC = GTFS_filepath+'SC/'
GTFS_SF = GTFS_filepath+'SF/'
GTFS_SL = GTFS_filepath+'SL/'
GTFS_SM = GTFS_filepath+'SM/'
GTFS_SO = GTFS_filepath+'SO/'
GTFS_SR = GTFS_filepath+'SR/'
GTFS_ST = GTFS_filepath+'ST/'
GTFS_TD = GTFS_filepath+'TD/'
GTFS_UC = GTFS_filepath+'UC/'
GTFS_VC = GTFS_filepath+'VC/'
GTFS_VN = GTFS_filepath+'VN/'
GTFS_VT = GTFS_filepath+'VT/'
GTFS_WC = GTFS_filepath+'WC/'
GTFS_WH = GTFS_filepath+'WH/'


# GTFS_3D2 = GTFS_filepath2+'3D/'
GTFS_AC2 = GTFS_filepath2+'AC/'
GTFS_AM2 = GTFS_filepath2+'AM/'
GTFS_AY2 = GTFS_filepath2+'AY/'
GTFS_BA2 = GTFS_filepath2+'BA/'
GTFS_Caltrain2 = GTFS_filepath2+'Caltrain/'
GTFS_CC2 = GTFS_filepath2+'CC/'
GTFS_CE2 = GTFS_filepath2+'CE/'
GTFS_CM2 = GTFS_filepath2+'CM/'
# GTFS_CT2 = GTFS_filepath2+'CT/'
GTFS_DE2 = GTFS_filepath2+'DE/'
GTFS_EM2 = GTFS_filepath2+'EM/'
GTFS_FF2 = GTFS_filepath2+'FF/'
GTFS_GG2 = GTFS_filepath2+'GG/'
GTFS_HF2 = GTFS_filepath2+'HF/'
GTFS_MA2 = GTFS_filepath2+'MA/'
GTFS_PE2 = GTFS_filepath2+'PE/'
GTFS_RV2 = GTFS_filepath2+'RV/'
GTFS_SB2 = GTFS_filepath2+'SB/'
GTFS_SC2 = GTFS_filepath2+'SC/'
GTFS_SF2 = GTFS_filepath2+'SF/'
GTFS_SL2 = GTFS_filepath2+'SL/'
GTFS_SM2 = GTFS_filepath2+'SM/'
GTFS_SO2 = GTFS_filepath2+'SO/'
GTFS_SR2 = GTFS_filepath2+'SR/'
GTFS_ST2 = GTFS_filepath2+'ST/'
GTFS_TD2 = GTFS_filepath2+'TD/'
GTFS_UC2 = GTFS_filepath2+'UC/'
GTFS_VC2 = GTFS_filepath2+'VC/'
GTFS_VN2 = GTFS_filepath2+'VN/'
GTFS_VT2 = GTFS_filepath2+'VT/'
GTFS_WC2 = GTFS_filepath2+'WC/'
GTFS_WH2 = GTFS_filepath2+'WH/'

GTFS_baseline = [
#     GTFS_3D,
                    GTFS_AC,
                    GTFS_AM,
                    GTFS_AY,
                    GTFS_BA,
                    GTFS_Caltrain,
                    GTFS_CC,
                    GTFS_CE,
                    GTFS_CM,
#                     GTFS_CT,
                    GTFS_DE,
                    GTFS_EM,
                    GTFS_FF,
                    GTFS_GG,
                    GTFS_HF,
                    GTFS_MA,
                    GTFS_PE,
                    GTFS_RV,
                    GTFS_SB,
                    GTFS_SC,
                    GTFS_SF,
                    GTFS_SL,
                    GTFS_SM,
                    GTFS_SO,
                    GTFS_SR,
                    GTFS_ST,
                    GTFS_TD,
                    GTFS_UC,
                    GTFS_VC,
                    GTFS_VN,
                    GTFS_VT,
                    GTFS_WC,
                    GTFS_WH]

GTFS_TR = [
#     GTFS_3D2,
                    GTFS_AC2,
                    GTFS_AM2,
                    GTFS_AY2,
                    GTFS_BA2,
                    GTFS_Caltrain2,
                    GTFS_CC2,
                    GTFS_CE2,
                    GTFS_CM2,
#                     GTFS_CT2,
                    GTFS_DE2,
                    GTFS_EM2,
                    GTFS_FF2,
                    GTFS_GG2,
                    GTFS_HF2,
                    GTFS_MA2,
                    GTFS_PE2,
                    GTFS_RV2,
                    GTFS_SB2,
                    GTFS_SC2,
                    GTFS_SF2,
                    GTFS_SL2,
                    GTFS_SM2,
                    GTFS_SO2,
                    GTFS_SR2,
                    GTFS_ST2,
                    GTFS_TD2,
                    GTFS_UC2,
                    GTFS_VC2,
                    GTFS_VN2,
                    GTFS_VT2,
                    GTFS_WC2,
                    GTFS_WH2]
GTFS_filepaths = [GTFS_baseline, GTFS_TR]



nrows = None

/Users/cpoliziani/opt/miniconda3/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:


#Ridership route
RR = pd.DataFrame()
#Ridership agency
RA = pd.DataFrame()

for fp, name, GTFS_filepath in zip(dataFilepath_sim,names, GTFS_filepaths):
    print('evaluate ridership')
    #import pathtraversal and person enter vehicles
    PT, PE = processEvents(fp, nrows)
    #filter PE transit trips from PT
    PE = filter_transit(PT, PE)
    #Guess transit agency for each PE
    PE = guess_agency(PE)
    #Guess transit route for each PE
    PE = guess_route(PE, GTFS_filepath)
    #Route Ridership
    rr = PE['route_id'].value_counts()
    sum_agency = 0
    sum_agency_bsva = 0
    sum_route = 0
    for route, count in zip(rr.keys(), rr):
        RR.at[route, name] = count
        sum_route += count
    ra = PE['agency'].value_counts()
    for agency, count in zip(ra.keys(), ra):
        RA.at[agency, name] = count
        sum_agency += count
        if agency in ['BA', 'SF', 'VT', 'AC']:
            RA.at[agency, name+' BA-SF-VT-AC'] = count
            sum_agency_bsva += count
    for route, count in zip(rr.keys(), rr):
        RR.at[route, name+' shares'] = count/sum_route
    ra = PE['agency'].value_counts()
    for agency, count in zip(ra.keys(), ra):
        RA.at[agency, name+' shares'] = count/sum_agency
        if agency in ['BA', 'SF', 'VT', 'AC']:
            RA.at[agency, name+' shares BA-SF-VT-AC'] = count/sum_agency_bsva
    RA.to_csv(output_filepath+RA_output)
    RR.to_csv(output_filepath+RR_output)

for year in years:
    diff = []
    diff_abs = []
    for baseline, future in zip(RA['Baseline'+year],RA['Future'+year]):
        diff.append((future-baseline)/baseline)
        diff_abs.append((future-baseline))
    RA['Diff %'+year] = diff
    RA['Diff'+year] = diff_abs

tot_NTD = 1756364558 + 15283299+5703705+49795740+110802986+7386518+49247910+27027693 + 8437926+50222832 + 2818648
    
RA.at['BA', 'clipper BA-SF-VT-AC 2016 share target'] = 350485/773719
RA.at['BA', 'MTC BA-SF-VT-AC 2016 share target'] = 458900/1564500
RA.at['BA', 'NTD BA-SF-VT-AC 2019 share target'] = (1756364558 + 15283299)/tot_NTD

RA.at['SF', 'clipper BA-SF-VT-AC 2016 share target'] = 293991/773719
RA.at['SF', 'MTC BA-SF-VT-AC 2016 share target'] = 777000/1564500
RA.at['SF', 'NTD BA-SF-VT-AC 2019 share target'] = (5703705+49795740+110802986+7386518+49247910)/tot_NTD

RA.at['VT', 'clipper BA-SF-VT-AC 2016 share target'] = 43950/773719
RA.at['VT', 'MTC BA-SF-VT-AC 2016 share target'] = 146700/1564500
RA.at['VT', 'NTD BA-SF-VT-AC 2019 share target'] = (27027693 + 8437926)/tot_NTD

RA.at['AC', 'clipper BA-SF-VT-AC 2016 share target'] = 85293/773719
RA.at['AC',
      'MTC BA-SF-VT-AC 2016 share target'] = 181900/1564500
RA.at['AC', 'NTD BA-SF-VT-AC 2019 share target'] = (50222832 + 2818648)/tot_NTD

RA.at['BA', 'clipper BA-SF-VT-AC 2020 Jan av ridership target'] = 350485
RA.at['BA', 'MTC BA-SF-VT-AC 2016 av ridership target'] = 458900
RA.at['BA', 'NTD BA-SF-VT-AC 2019 ridership target'] = (1756364558 + 15283299)

RA.at['SF', 'clipper BA-SF-VT-AC 2020 Jan av ridership target'] = 293991
RA.at['SF', 'MTC BA-SF-VT-AC 2016 av ridership target'] = 777000
RA.at['SF', 'NTD BA-SF-VT-AC 2019 ridership target'] = (5703705+49795740+110802986+7386518+49247910)

RA.at['VT', 'clipper BA-SF-VT-AC 2020 Jan av ridership target'] = 43950
RA.at['VT', 'MTC BA-SF-VT-AC 2016 av ridership target'] = 146700
RA.at['VT', 'NTD BA-SF-VT-AC 2019 ridership target'] =  (27027693 + 8437926)

RA.at['AC', 'clipper BA-SF-VT-AC 2020 Jan av ridership target'] = 85293
RA.at['AC', 'MTC BA-SF-VT-AC 2016 av ridership target'] = 181900
RA.at['AC', 'NTD BA-SF-VT-AC 2019 ridership target'] = (50222832 + 2818648)

RA.to_csv(output_filepath+RA_output)
RR.to_csv(output_filepath+RR_output)




evaluate ridership
read s3://beam-outputs/pilates-outputs/sfbay_baseline_20230224/beam/year-2019-iteration-5/ITERS/it.0/0.events.csv.gz


/var/folders/h9/g8d27p9s5js3dx9z7b5y8k5c0000gr/T/ipykernel_32884/1144239604.py:13: DtypeWarning: Columns (0,2,7,8,12,13,15,16,29,31,32,34,39,41,43,44,45,46,47,49,62) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(dataFilepath_sim, compression = 'gzip',chunksize=1500000, nrows = nrows):


ParkingEvent            817746
PathTraversal           165932
PersonEntersVehicle     126177
departure                91877
PersonLeavesVehicle      61520
ModeChoice               47953
actend                   47790
LeavingParkingEvent      38552
actstart                 33824
arrival                  33824
PersonCost               30435
TeleportationEvent        3512
ReserveRideHail            724
Replanning                 131
ChargingPlugInEvent          1
RefuelSessionEvent           1
ChargingPlugOutEvent         1
Name: type, dtype: int64


/var/folders/h9/g8d27p9s5js3dx9z7b5y8k5c0000gr/T/ipykernel_32884/1144239604.py:13: DtypeWarning: Columns (14,16,34,47,62) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(dataFilepath_sim, compression = 'gzip',chunksize=1500000, nrows = nrows):


PathTraversal           389318
PersonEntersVehicle     187022
PersonLeavesVehicle     158425
ModeChoice              113271
departure               112661
actend                  112655
arrival                  91698
actstart                 91672
LeavingParkingEvent      82191
PersonCost               76150
ParkingEvent             67805
TeleportationEvent       14358
ReserveRideHail           2118
Replanning                 650
ChargingPlugInEvent          2
RefuelSessionEvent           2
ChargingPlugOutEvent         2
Name: type, dtype: int64


...

PathTraversal           380126
PersonLeavesVehicle     179469
PersonEntersVehicle     170436
arrival                 106499
actstart                106487
ModeChoice              102983
departure               101232
actend                  101231
PersonCost               81552
LeavingParkingEvent      76212
ParkingEvent             73835
TeleportationEvent       15553
ReserveRideHail           2462
Replanning                1908
RefuelSessionEvent           5
ChargingPlugOutEvent         5
ChargingPlugInEvent          5
Name: type, dtype: int64


/var/folders/h9/g8d27p9s5js3dx9z7b5y8k5c0000gr/T/ipykernel_32884/1144239604.py:13: DtypeWarning: Columns (1,12,42) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(dataFilepath_sim, compression = 'gzip',chunksize=1500000, nrows = nrows):


PathTraversal           396064
PersonLeavesVehicle     188545
PersonEntersVehicle     160260
actstart                112228
arrival                 112218
ModeChoice               94539
departure                93080
actend                   93079
PersonCost               83463
ParkingEvent             75789
LeavingParkingEvent      72072
TeleportationEvent       15398
ReserveRideHail           1819
Replanning                1419
RefuelSessionEvent           9
ChargingPlugOutEvent         9
ChargingPlugInEvent          9
Name: type, dtype: int64


/var/folders/h9/g8d27p9s5js3dx9z7b5y8k5c0000gr/T/ipykernel_32884/1144239604.py:13: DtypeWarning: Columns (1,12,30,35,42) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(dataFilepath_sim, compression = 'gzip',chunksize=1500000, nrows = nrows):


PathTraversal           147022
PersonLeavesVehicle      63414
PersonEntersVehicle      43307
actstart                 38804
arrival                  38802
PersonCost               25553
ParkingEvent             24153
ModeChoice               23606
actend                   23520
departure                23520
LeavingParkingEvent      21309
TeleportationEvent        3126
ReserveRideHail            267
Replanning                 149
RefuelSessionEvent           3
ChargingPlugOutEvent         3
ChargingPlugInEvent          2
Name: type, dtype: int64
         vehicle    time
887249    264985   18003
887250    774284   18000
887251    100246   18003
887252    724126   18007
887254    224200   18000
...          ...     ...
36476286  560092  186420
36476323  251000  187237
36476377  560092  191020
36476384  251000  191168
36476400  251000  192097

[2055122 rows x 2 columns]
car          2652686
bus          1291011
walk         1257271
car_hov2      423832
car_hov3      301741
tram           

In [3]:


RA


,Baseline2019,Baseline2019 BA-SF-VT-AC,Baseline2019 shares,Baseline2019 shares BA-SF-VT-AC,Future2019,Future2019 BA-SF-VT-AC,Future2019 shares,Future2019 shares BA-SF-VT-AC,Diff %2019,Diff2019,clipper BA-SF-VT-AC 2016 share target,MTC BA-SF-VT-AC 2016 share target,NTD BA-SF-VT-AC 2019 share target,clipper BA-SF-VT-AC 2020 Jan av ridership target,MTC BA-SF-VT-AC 2016 av ridership target,NTD BA-SF-VT-AC 2019 ridership target
SF,97122.0,97122.0,0.324144,0.404953,97939.0,97939.0,0.327439,0.408254,0.008412,817.0,0.379971,0.496644,0.107022,293991.0,777000.0,2.229369e+08
AC,57855.0,57855.0,0.193091,0.241228,57347.0,57347.0,0.191728,0.239048,-0.008781,-508.0,0.110238,0.116267,0.025463,85293.0,181900.0,5.304148e+07
VT,46949.0,46949.0,0.156692,0.195755,47706.0,47706.0,0.159495,0.198860,0.016124,757.0,0.056804,0.093768,0.017025,43950.0,146700.0,3.546562e+07
BA,37909.0,37909.0,0.126521,0.158063,36905.0,36905.0,0.123384,0.153837,-0.026484,-1004.0,0.452987,0.293321,0.850490,350485.0,458900.0,1.771648e+09
ST,17101.0,NaN,0.057074,NaN,16897.0,NaN,0.056492,NaN,-0.011929,-204.0,NaN,NaN,NaN,NaN,NaN,NaN
CC,8185.0,NaN,0.027317,NaN,7910.0,NaN,0.026445,NaN,-0.033598,-275.0,NaN,NaN,NaN,NaN,NaN,NaN
GG,7802.0,NaN,0.026039,NaN,7543.0,NaN,0.025218,NaN,-0.033197,-259.0,NaN,NaN,NaN,NaN,NaN,NaN
TD,4291.0,NaN,0.014321,NaN,4167.0,NaN,0.013932,NaN,-0.028898,-124.0,NaN,NaN,NaN,NaN,NaN,NaN
SR,2766.0,NaN,0.009232,NaN,2834.0,NaN,0.009475,NaN,0.024584,68.0,NaN,NaN,NaN,NaN,NaN,NaN
WH,2675.0,NaN,0.008928,NaN,2766.0,NaN,0.009248,NaN,0.034019,91.0,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
RR[RR.index=='SF:18608']

,Baseline2019,Baseline2019 shares,Future2019,Future2019 shares
SF:18608,NaN,NaN,4607.0,0.015403


In [5]:
RR[RR.index=='SF:12327']

,Baseline2019,Baseline2019 shares,Future2019,Future2019 shares
SF:12327,2993.0,0.009989,NaN,NaN


In [6]:

RR[RR.index.str.contains('AC:1-142')]


,Baseline2019,Baseline2019 shares,Future2019,Future2019 shares
AC:1-142,2226.0,0.007429,NaN,NaN


In [7]:

RR[RR.index.str.contains('AC:1T-142')]


,Baseline2019,Baseline2019 shares,Future2019,Future2019 shares
AC:1T-142,NaN,NaN,2729.0,0.009124


In [8]:



RR[RR.index.str.contains('CA:')]



,Baseline2019,Baseline2019 shares,Future2019,Future2019 shares
CA:12868,1105.0,0.003688,1276.0,0.004266
CA:12869,661.0,0.002206,889.0,0.002972
CA:12867,330.0,0.001101,355.0,0.001187


In [9]:


RR[RR.index.str.contains('SF:1000')]


,Baseline2019,Baseline2019 shares,Future2019,Future2019 shares
SF:1000,NaN,NaN,1888.0,0.006312


In [10]:

#The Central Subway Project will improve public transportation in San Francisco by extending the Muni Metro T Third Line through SoMa

#J
RR[RR.index.str.contains('SF:12475')]


,Baseline2019,Baseline2019 shares,Future2019,Future2019 shares
SF:12475,2142.0,0.007149,2084.0,0.006967


In [11]:

#K/T
RR[RR.index.str.contains('SF:12476')]


,Baseline2019,Baseline2019 shares,Future2019,Future2019 shares
SF:12476,4349.0,0.014515,4342.0,0.014517


In [12]:

#L
RR[RR.index.str.contains('SF:12477')]


,Baseline2019,Baseline2019 shares,Future2019,Future2019 shares
SF:12477,2836.0,0.009465,3069.0,0.010261


In [13]:

#M
RR[RR.index.str.contains('SF:12478')]


,Baseline2019,Baseline2019 shares,Future2019,Future2019 shares
SF:12478,2848.0,0.009505,2900.0,0.009696


In [14]:

#N
RR[RR.index.str.contains('SF:12479')]



,Baseline2019,Baseline2019 shares,Future2019,Future2019 shares
SF:12479,4698.0,0.01568,4566.0,0.015265


In [15]:
for year in years:
    
    print('total SF light rail baseline'+year,
         (list(RR[RR.index.str.contains('SF:12475')]['Baseline'+year])[0]+
          list(RR[RR.index.str.contains('SF:12476')]['Baseline'+year])[0]+
          list(RR[RR.index.str.contains('SF:12477')]['Baseline'+year])[0]+
          list(RR[RR.index.str.contains('SF:12478')]['Baseline'+year])[0]+
          list(RR[RR.index.str.contains('SF:12479')]['Baseline'+year])[0])
         )
    print('total SF light rail future'+year,
         (list(RR[RR.index.str.contains('SF:12475')]['Future'+year])[0]+
          list(RR[RR.index.str.contains('SF:12476')]['Future'+year])[0]+
          list(RR[RR.index.str.contains('SF:12477')]['Future'+year])[0]+
          list(RR[RR.index.str.contains('SF:12478')]['Future'+year])[0]+
          list(RR[RR.index.str.contains('SF:12479')]['Future'+year])[0]+
          list(RR[RR.index.str.contains('SF:1000')]['Future'+year])[0])
         )

total SF light rail baseline2019 16873.0
total SF light rail future2019 18849.0


In [3]:
#Imports
import numpy as np
import pandas as pd
import difflib
import matplotlib.pyplot as plt
import time

#########Analysis of what agents used in the baseline
index_baseline =0 # to update
index_future =1
fp_PtoPTss_baseline = '/Users/cpoliziani/Downloads/Transit Rich/data_year2018it6/it52018/personToVehicles_baseline_it5_2018.csv.gz' #to Update
fp_PtoPTss_future = '/Users/cpoliziani/Downloads/Transit Rich/data_year2018it6/it52018/personToVehicles_TR_it5_2018.csv.gz' #to Update

# print('read PtoPTss baseline',fp_PtoPTss_baseline)
PtoPTss_baseline = pd.read_csv(fp_PtoPTss_baseline)
# print('read PtoPTss TR', fp_PtoPTss_future)
PtoPTss_TR = pd.read_csv(fp_PtoPTss_future)
# print('read plans baseline',dataFilepath_sim[index_baseline][:-13]+'plans.csv.gz')
plans_baseline = pd.read_csv(dataFilepath_sim[index_baseline][:-13]+'plans.csv.gz')
# print('read plans TR', dataFilepath_sim[index_future][:-13]+'plans.csv.gz')
plans_TR = pd.read_csv(dataFilepath_sim[index_future][:-13]+'plans.csv.gz')
# print('read GTFS trips fpr AC, SF and CA')
line_1T_trips_TR = pd.read_csv(GTFS_AC2+'trips.txt')
line_1T_trips_TR = line_1T_trips_TR[line_1T_trips_TR['route_id'].str.contains('1T-142')]
line_CS_trips_TR = pd.read_csv(GTFS_SF2+'trips.txt')
line_CS_trips_TR = line_CS_trips_TR[(line_CS_trips_TR['route_id']==1000)]
line_VN_trips_TR = pd.read_csv(GTFS_SF2+'trips.txt')
line_VN_trips_TR = line_VN_trips_TR[(line_VN_trips_TR['route_id']==18608)]
line_CA_trips_TR = pd.read_csv(GTFS_Caltrain2+'trips.txt')


/var/folders/h9/g8d27p9s5js3dx9z7b5y8k5c0000gr/T/ipykernel_54143/453992257.py:15: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  PtoPTss_baseline = pd.read_csv(fp_PtoPTss_baseline)
/var/folders/h9/g8d27p9s5js3dx9z7b5y8k5c0000gr/T/ipykernel_54143/453992257.py:17: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  PtoPTss_TR = pd.read_csv(fp_PtoPTss_future)


In [6]:
events_baseline = pd.read_csv(dataFilepath_sim[index_baseline])
events_TR = pd.read_csv(dataFilepath_sim[index_future])
mode_choice_baseline = events_baseline[events_baseline['type']=='ModeChoice']
mode_choice_TR = events_TR[events_TR['type']=='ModeChoice']
PTs_baseline = events_baseline[events_baseline['type']=='PathTraversal']
PTs_TR = events_TR[events_TR['type']=='PathTraversal']

/var/folders/h9/g8d27p9s5js3dx9z7b5y8k5c0000gr/T/ipykernel_54143/66457010.py:1: DtypeWarning: Columns (0,1,6,7,12,18,30,32,33,34,35,39,40,41,42,52,55,57,59,60,61,62) have mixed types. Specify dtype option on import or set low_memory=False.
  events_baseline = pd.read_csv(dataFilepath_sim[index_baseline])
/var/folders/h9/g8d27p9s5js3dx9z7b5y8k5c0000gr/T/ipykernel_54143/66457010.py:2: DtypeWarning: Columns (0,1,2,3,4,5,6,7,11,24,29,32,33,40,43,45,47,48,49,50,52,55) have mixed types. Specify dtype option on import or set low_memory=False.
  events_TR = pd.read_csv(dataFilepath_sim[index_future])


In [7]:
PtoPTss_AC_TR = PtoPTss_TR[PtoPTss_TR['vehicle2'].str.contains('AC', na=False)]
PtoPTss_SF_TR = PtoPTss_TR[PtoPTss_TR['vehicle2'].str.contains('SF', na=False)]
PtoPTss_VN_TR = PtoPTss_TR[PtoPTss_TR['vehicle2'].str.contains('SF', na=False)]
PtoPTss_CA_TR = PtoPTss_TR[PtoPTss_TR['vehicle2'].str.contains('Ca', na=False)]

#Filter SF trips - CS only
is_CS = []
vehicles = list(line_CS_trips_TR['trip_id'])
for vehicle in PtoPTss_SF_TR['vehicleID']:
    if int(vehicle[3:]) in vehicles:
        is_CS.append(True)
    else:
        is_CS.append(False)
print('Filter CS trips [tot, remained]',len(is_CS),sum(is_CS))
PtoPTss_SF_TR['is_CS'] = is_CS
PtoPTss_SF_TR = PtoPTss_SF_TR[PtoPTss_SF_TR['is_CS']==True]

is_VN = []
vehicles = list(line_VN_trips_TR['trip_id'])
for vehicle in PtoPTss_VN_TR['vehicleID']:
    if int(vehicle[3:]) in vehicles:
        is_VN.append(True)
    else:
        is_VN.append(False)
print('Filter CS trips [tot, remained]',len(is_VN),sum(is_VN))
PtoPTss_VN_TR['is_VN'] = is_VN
PtoPTss_VN_TR = PtoPTss_VN_TR[PtoPTss_VN_TR['is_VN']==True]

is_AC = []
vehicles = list(line_1T_trips_TR['trip_id'])
for vehicle in PtoPTss_AC_TR['vehicleID']:
    if int(vehicle[3:]) in vehicles:
        is_AC.append(True)
    else:
        is_AC.append(False)
print('Filter 1T trips [tot, remained]',len(is_AC),sum(is_AC))
PtoPTss_AC_TR['is_AC'] = is_AC
PtoPTss_AC_TR = PtoPTss_AC_TR[PtoPTss_AC_TR['is_AC']==True]

Filter CS trips [tot, remained] 632364 4226


/var/folders/h9/g8d27p9s5js3dx9z7b5y8k5c0000gr/T/ipykernel_54143/3649058896.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PtoPTss_SF_TR['is_CS'] = is_CS


Filter CS trips [tot, remained] 632364 18563


/var/folders/h9/g8d27p9s5js3dx9z7b5y8k5c0000gr/T/ipykernel_54143/3649058896.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PtoPTss_VN_TR['is_VN'] = is_VN


Filter 1T trips [tot, remained] 557759 16150


/var/folders/h9/g8d27p9s5js3dx9z7b5y8k5c0000gr/T/ipykernel_54143/3649058896.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PtoPTss_AC_TR['is_AC'] = is_AC


In [8]:
dict_time_baseline = dict(zip(PTs_baseline.index, PTs_baseline.time))
dict_time_TR = dict(zip(PTs_TR.index, PTs_TR.time))

times = []
for pid in PtoPTss_baseline['pathTraversalID']:
    times.append(dict_time_baseline[pid])
PtoPTss_baseline['time'] = times


times = []
for pid in PtoPTss_TR['pathTraversalID']:
    times.append(dict_time_TR[pid])
PtoPTss_TR['time'] = times

In [9]:
times = []
for pid in PtoPTss_CA_TR['pathTraversalID']:
    times.append(dict_time_TR[pid])
PtoPTss_CA_TR['time'] = times

times = []
for pid in PtoPTss_SF_TR['pathTraversalID']:
    times.append(dict_time_TR[pid])
PtoPTss_SF_TR['time'] = times

times = []
for pid in PtoPTss_VN_TR['pathTraversalID']:
    times.append(dict_time_TR[pid])
PtoPTss_VN_TR['time'] = times

times = []
for pid in PtoPTss_AC_TR['pathTraversalID']:
    times.append(dict_time_TR[pid])
PtoPTss_AC_TR['time'] = times

/var/folders/h9/g8d27p9s5js3dx9z7b5y8k5c0000gr/T/ipykernel_54143/1726032748.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PtoPTss_CA_TR['time'] = times


In [10]:
persons_baseline = pd.DataFrame()
i = 0
trips_TR = pd.concat([line_CA_trips_TR['trip_id'],line_CS_trips_TR['trip_id'],line_1T_trips_TR['trip_id'],line_VN_trips_TR['trip_id']])
##############################
# PtoPTss_TR2=pd.concat([PtoPTss_AC_TR,PtoPTss_SF_TR,PtoPTss_CA_TR])
# PtoPTss_TR2_dict = PtoPTss.groupby(['personID', 'planIndex']).apply(lambda x: [sum(x.length),
#                                                     sum(x.duration)]).to_dict()
# print(len(PtoPTss_CA_TR))
PtoPTss_TR_CA_dict = PtoPTss_CA_TR.groupby(['personID', 'planIndex']).apply(lambda x: [sum(x.length),
                                                    sum(x.duration),min(x.time)]).to_dict()
# print(len(PtoPTss_TR_CA_dict.keys()))
# print(len(PtoPTss_SF_TR))
PtoPTss_TR_SF_dict = PtoPTss_SF_TR.groupby(['personID', 'planIndex']).apply(lambda x: [sum(x.length),
                                                    sum(x.duration),min(x.time)]).to_dict()
PtoPTss_TR_VN_dict = PtoPTss_VN_TR.groupby(['personID', 'planIndex']).apply(lambda x: [sum(x.length),
                                                    sum(x.duration),min(x.time)]).to_dict()
# print(len(PtoPTss_TR_SF_dict.keys()))
# print(len(PtoPTss_AC_TR))
PtoPTss_TR_AC_dict = PtoPTss_AC_TR.groupby(['personID', 'planIndex']).apply(lambda x: [sum(x.length),
                                                    sum(x.duration),min(x.time)]).to_dict()
# print(len(PtoPTss_TR_AC_dict.keys()))
# print(len(PtoPTss_baseline))
PtoPTss_baseline_dict = PtoPTss_baseline.groupby(['personID', 'planIndex']).apply(lambda x: [sum(x.length),
                                                    sum(x.duration),min(x.time)]).to_dict()
# print(len(PtoPTss_baseline_dict.keys()))
# print(len(PtoPTss_TR))
PtoPTss_TR_dict = PtoPTss_TR.groupby(['personID', 'planIndex']).apply(lambda x: [sum(x.length),
                                                sum(x.duration),min(x.time)]).to_dict()
print(len(PtoPTss_TR_dict.keys()))


#Add logsum
# person_trips_baseline = pd.read_csv('/Users/cpoliziani/Downloads/tourTripsMerged_Baseline.csv') #to Update
# person_trips_baseline['trip_num']=person_trips_baseline['trip_num']*2-1
# person_trips_TR = pd.read_csv('/Users/cpoliziani/Downloads/tourTripsMerged_TR.csv')  #to Update
# person_trips_TR['trip_num']=person_trips_TR['trip_num']*2-1

2024123


In [11]:
person_trips_baseline = pd.read_csv('/Users/cpoliziani/Downloads/Transit Rich/data_year2018it6/it52018/sfbay_baseline_20230224_inexus_it52018.csv') #to Update
person_trips_TR = pd.read_csv('/Users/cpoliziani/Downloads/Transit Rich/data_year2018it6/it52018/sfbay_baseline_TR_20230224_inexus_it52018.csv') #to Update
# logsum_trip_Potential_INEXUS
# destination_logsum_x
# destination_logsum_y
# logsum_tours_mode_AS_tours
# workplace_location_logsum



/var/folders/h9/g8d27p9s5js3dx9z7b5y8k5c0000gr/T/ipykernel_54143/1801712134.py:1: DtypeWarning: Columns (52) have mixed types. Specify dtype option on import or set low_memory=False.
  person_trips_baseline = pd.read_csv('/Users/cpoliziani/Downloads/Transit Rich/data_year2018it6/it52018/sfbay_baseline_20230224_inexus_it52018.csv') #to Update
/var/folders/h9/g8d27p9s5js3dx9z7b5y8k5c0000gr/T/ipykernel_54143/1801712134.py:2: DtypeWarning: Columns (52) have mixed types. Specify dtype option on import or set low_memory=False.
  person_trips_TR = pd.read_csv('/Users/cpoliziani/Downloads/Transit Rich/data_year2018it6/it52018/sfbay_baseline_TR_20230224_inexus_it52018.csv') #to Update


In [12]:
persons_baseline = pd.DataFrame()
persons_baseline = pd.DataFrame(columns = ['Person',
                                           'Plan Index',
                                           'Planned Depart Time Baseline',
                                           'Planned Depart Time TR',
                                           'Activity From Baseline',
                                           'Activity To Baseline',
                                           'Activity From TR',
                                           'Activity To TR',
                                           'Project Tried',
                                           'Vehicles Used Baseline',
                                           'Vehicle Types Used Baseline',                                          
                                           'Bus agencies Used Baseline',
                                           'First Bus agency Used Baseline',
                                           'Vehicles Used TR',
                                           'Vehicle Types Used TR',                                          
                                           'Bus agencies Used TR',
                                           'First Bus agency Used TR',
                                           'Switch From',
                                           'Trip Length Baseline',                       
                                           'Trip Duration Baseline',
                                           'Trip Length TR',
                                           'Trip Duration TR',
                                           'Diff Length',
                                           'Diff Duration',
                                           'Planned Mode Baseline',                                         
                                           'Planned Mode TR',                                          
                                           'Chosen Mode Baseline',                                          
                                           'Chosen Mode TR',
                                           'X Activity From TR',
                                           'Y Activity From TR',
                                           'X Activity To TR',
                                           'Y Activity To TR',
                                           'X Activity From Baseline',
                                           'Y Activity From Baseline',
                                           'X Activity To Baseline',
                                           'Y Activity To Baseline',
                                           'Log Sum Baseline',
                                           'Log Sum TR',
                                           'Diff Log Sum',
#                                            'ZIP Departure TR',                                          
#                                            'ZIP Arrival TR',                                          
#                                            'ZIP Departure Baseline',                                          
#                                            'ZIP Arrival Baseline',                                          
                                          ])

PtoPTss_baseline_bus =  PtoPTss_baseline[(PtoPTss_baseline['mode'] =='bus')|
                                         (PtoPTss_baseline['mode'] =='subway')|
                                         (PtoPTss_baseline['mode'] =='tram')|
                                         (PtoPTss_baseline['mode'] =='rail')|
                                         (PtoPTss_baseline['mode'] =='cable_car')|
                                         (PtoPTss_baseline['mode'] =='ferry')]
PtoPTss_TR_bus =  PtoPTss_TR[(PtoPTss_TR['mode'] =='bus')|
                                         (PtoPTss_TR['mode'] =='subway')|
                                         (PtoPTss_TR['mode'] =='tram')|
                                         (PtoPTss_TR['mode'] =='rail')|
                                         (PtoPTss_TR['mode'] =='cable_car')|
                                         (PtoPTss_TR['mode'] =='ferry')]   
print(len(PtoPTss_TR_SF_dict))
print(len(PtoPTss_TR_VN_dict))
print(len(PtoPTss_TR_CA_dict))
print(len(PtoPTss_TR_AC_dict))


i=0      
for row in PtoPTss_TR_SF_dict.keys():
    i+=1
    if i%50==0:
        print(i)
        
    try:
        persons_baseline.at[i,'Person'] = row[0]
    except:
        print('Warning', row, 'person')
    try:
        persons_baseline.at[i,'Plan Index'] = int(row[1])
    except:
        print('Warning', row, 'plan_index')
    try:
        persons_baseline.at[i,'Activity From Baseline'] = list(plans_baseline['activityType'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]-1)])[0]
    except:
        print('Warning', row, 'Activity From Baseline')
    try:
        persons_baseline.at[i,'Activity To Baseline'] = list(plans_baseline['activityType'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]+1)])[0]
    except:
        print('Warning', row, 'Activity To Baseline')
    try:
        persons_baseline.at[i,'Activity From TR'] = list(plans_TR['activityType'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]-1)])[0]
    except:
        print('Warning', row, 'Activity From TR')
    try:
        persons_baseline.at[i,'Activity To TR'] = list(plans_TR['activityType'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]+1)])[0]
    except:
        print('Warning', row, 'Activity To TR')  
    try:
        persons_baseline.at[i,'Planned Depart Time Baseline'] = int(list(plans_baseline['activityEndTime'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]-1)])[0])
    except:
        print('Warning', row, 'plan_dep_time_baseline')
    try:
        persons_baseline.at[i,'Planned Depart Time TR'] = int(list(plans_TR['activityEndTime'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]-1)])[0])
    except:
        print('Warning', row, 'plan_dep_time_TR')
    try:
        persons_baseline.at[i,'Project Tried'] = 'SF - Central Subway'
    except:
        print('Warning', row, 'project tried')
    try:
        persons_baseline.at[i,'Potential INEXUS Baseline'] = np.mean(list(person_trips_baseline['logsum_trip_Potential_INEXUS'][(person_trips_baseline['person_id']==row[0])]))
    except:
        print('Warning', row, 'Potential INEXUS Baseline')
    try:
        persons_baseline.at[i,'Potential INEXUS TR'] = np.mean(list(person_trips_TR['logsum_trip_Potential_INEXUS'][(person_trips_TR['person_id']==row[0])]))
    except:
        print('Warning', row, 'Potential INEXUS TR')   
    try:
        persons_baseline.at[i,'Destination LOG SUM X Baseline'] = np.mean(list(person_trips_baseline['destination_logsum_x'][(person_trips_baseline['person_id']==row[0])]))
    except:
        print('Warning', row, 'Destination LOG SUM X Baseline')
    try:
        persons_baseline.at[i,'Destination LOG SUM X TR'] = np.mean(list(person_trips_TR['destination_logsum_x'][(person_trips_TR['person_id']==row[0])]))
    except:
        print('Warning', row, 'Destination LOG SUM X TR')
    try:
        persons_baseline.at[i,'Destination LOG SUM Y Baseline'] = np.mean(list(person_trips_baseline['destination_logsum_y'][(person_trips_baseline['person_id']==row[0])]))
    except:
        print('Warning', row, 'Destination LOG SUM Y Baseline')
    try:
        persons_baseline.at[i,'Destination LOG SUM Y TR'] = np.mean(list(person_trips_TR['destination_logsum_y'][(person_trips_TR['person_id']==row[0])]))
    except:
        print('Warning', row, 'Destination LOG SUM Y TR')
    try:
        persons_baseline.at[i,'LOG SUM Tours Mode AS Baseline'] = np.mean(list(person_trips_baseline['logsum_tours_mode_AS_tours'][(person_trips_baseline['person_id']==row[0])]))
    except:
        print('Warning', row, 'LOG SUM Tours Mode AS Baseline')
    try:
        persons_baseline.at[i,'LOG SUM Tours Mode AS TR'] = np.mean(list(person_trips_TR['logsum_tours_mode_AS_tours'][(person_trips_TR['person_id']==row[0])]))
    except:
        print('Warning', row, 'LOG SUM Tours Mode AS TR')
    try:
        persons_baseline.at[i,'LOG SUM Work PLace Location Baseline'] = np.mean(list(person_trips_baseline['workplace_location_logsum'][(person_trips_baseline['person_id']==row[0])]))
    except:
        print('Warning', row, 'LOG SUM Work PLace Location Baseline')
    try:
        persons_baseline.at[i,'LOG SUM Work PLace Location TR'] = np.mean(list(person_trips_TR['workplace_location_logsum'][(person_trips_TR['person_id']==row[0])]))
    except:
        print('Warning', row, 'LOG SUM Work PLace Location TR')     
    try:
        persons_baseline.at[i,'Diff Potential INEXUS'] = persons_baseline.at[i,'Potential INEXUS TR']-persons_baseline.at[i,'Potential INEXUS Baseline']
    except:
        print('Warning', row, 'Diff Potential INEXUS')
    try:
        persons_baseline.at[i,'Diff Destination LOG SUM X'] = persons_baseline.at[i,'Destination LOG SUM X TR']-persons_baseline.at[i,'Destination LOG SUM X Baseline']
    except:
        print('Warning', row, 'Diff Destination LOG SUM X')
    try:
        persons_baseline.at[i,'Diff Destination LOG SUM Y'] = persons_baseline.at[i,'Destination LOG SUM Y TR']-persons_baseline.at[i,'Destination LOG SUM Y Baseline']
    except:
        print('Warning', row, 'Diff Destination LOG SUM Y')
    try:
        persons_baseline.at[i,'Diff LOG SUM Tours Mode AS'] = persons_baseline.at[i,'LOG SUM Tours Mode AS TR']-persons_baseline.at[i,'LOG SUM Tours Mode AS Baseline']
    except:
        print('Warning', row, 'Diff LOG SUM Tours Mode AS')
    try:
        persons_baseline.at[i,'Diff LOG SUM Work PLace Location'] = persons_baseline.at[i,'LOG SUM Work PLace Location TR']-persons_baseline.at[i,'LOG SUM Work PLace Location Baseline']
    except:
        print('Warning', row, 'Diff LOG SUM Work PLace Location')
    try:
        persons_baseline.at[i,'Vehicles Used Baseline'] = np.unique(list(PtoPTss_baseline['vehicleID'][(PtoPTss_baseline['personID']==row[0])&(PtoPTss_baseline['planIndex']==row[1])]))
    except:
        print('Warning', row, 'Vehicles Used Baseline')
    try:
        persons_baseline.at[i,'Vehicles Used TR'] = np.unique(list(PtoPTss_TR['vehicleID'][(PtoPTss_TR['personID']==row[0])&(PtoPTss_TR['planIndex']==row[1])]))
    except:
        print('Warning', row, 'Vehicles Used TR')
    try:
        persons_baseline.at[i,'Bus agencies Used Baseline'] = np.unique(list(PtoPTss_baseline_bus['vehicle2'][(PtoPTss_baseline_bus['personID']==row[0])&(PtoPTss_baseline_bus['planIndex']==row[1])]))
    except:
        print('Warning', row, 'Bus agencies Used Baseline')
    try:
        persons_baseline.at[i,'Bus agencies Used TR'] = np.unique(list(PtoPTss_TR_bus['vehicle2'][(PtoPTss_TR_bus['personID']==row[0])&(PtoPTss_TR_bus['planIndex']==row[1])]))
    except:
        print('Warning', row, 'Bus agencies Used TR') 
    try:
        persons_baseline.at[i,'First Bus agency Used Baseline'] = np.unique(list(PtoPTss_baseline_bus['vehicle2'][(PtoPTss_baseline_bus['personID']==row[0])&(PtoPTss_baseline_bus['planIndex']==row[1])]))[0]
    except:
        print('Warning', row, 'First Bus agency Used Baseline')
    try:
        persons_baseline.at[i,'First Bus agency Used TR'] = np.unique(list(PtoPTss_TR_bus['vehicle2'][(PtoPTss_TR_bus['personID']==row[0])&(PtoPTss_TR_bus['planIndex']==row[1])]))[0]
    except:
        print('Warning', row, 'First Bus agency Used TR')
    try:
        persons_baseline.at[i,'Trip Length TR'] = int(PtoPTss_TR_dict[row][0])
    except:
        print('Warning', row, 'TR_length')
    try:
        persons_baseline.at[i,'Trip Duration TR'] = int(PtoPTss_TR_dict[row][1])
    except:
        print('Warning', row, 'TR_duration')
    try:
        persons_baseline.at[i,'Trip Length Baseline'] = int(PtoPTss_baseline_dict[row][0])
    except:
        print('Warning', row, 'baseline_length')
    try:
        persons_baseline.at[i,'Trip Duration Baseline'] = int(PtoPTss_baseline_dict[row][1])
    except:
        print('Warning', row, 'baseline_duration')
    try:
        persons_baseline.at[i,'Diff Length'] = int(persons_baseline.at[i,'Trip Length TR']-persons_baseline.at[i,'Trip Length Baseline'])
    except:
        print('Warning', row, 'diff_length')
    try:
        persons_baseline.at[i,'Diff Duration'] = int(persons_baseline.at[i,'Trip Duration TR']-persons_baseline.at[i,'Trip Duration Baseline'])
    except:
        print('Warning', row, 'diff_duration')
    try:
        persons_baseline.at[i,'X Activity From TR'] = list(plans_TR['activityLocationX'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]-1)])[0]
    except:
        print('Warning', row, 'x_activity_from_TR')
    try:
        persons_baseline.at[i,'Y Activity From TR'] = list(plans_TR['activityLocationY'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]-1)])[0]
    except:
        print('Warning', row, 'y_activity_From_TR')
    try:
        persons_baseline.at[i,'X Activity To TR'] = list(plans_TR['activityLocationX'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]+1)])[0]
    except:
        print('Warning', row, 'x_activity_To_TR')
    try:
        persons_baseline.at[i,'Y Activity To TR'] = list(plans_TR['activityLocationY'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]+1)])[0]
    except:
        print('Warning', row, 'y_activity_To_TR')   
    try:
        persons_baseline.at[i,'X Activity From Baseline'] = list(plans_baseline['activityLocationX'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]-1)])[0]
    except:
        print('Warning', row, 'x_activity_from_Baseline')
    try:
        persons_baseline.at[i,'Y Activity From Baseline'] = list(plans_baseline['activityLocationY'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]-1)])[0]
    except:
        print('Warning', row, 'y_activity_From_Baseline')
    try:
        persons_baseline.at[i,'X Activity To Baseline'] = list(plans_baseline['activityLocationX'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]+1)])[0]
    except:
        print('Warning', row, 'x_activity_To_Baseline')
    try:
        persons_baseline.at[i,'Y Activity To Baseline'] = list(plans_baseline['activityLocationY'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]+1)])[0]
    except:
        print('Warning', row, 'y_activity_To_Baseline')   
    try:
        persons_baseline.at[i,'Vehicle Types Used Baseline'] = np.unique(list(PtoPTss_baseline['mode'][(PtoPTss_baseline['personID']==row[0])&(PtoPTss_baseline['planIndex']==row[1])]))
    except:
        print('Warning', row, 'Vehicle Types Used baseline')
    try:
        persons_baseline.at[i,'Vehicle Types Used TR'] = np.unique(list(PtoPTss_TR['mode'][(PtoPTss_TR['personID']==row[0])&(PtoPTss_TR['planIndex']==row[1])]))
    except:
        print('Warning', row, 'Vehicle Types Used TR')
    try:
        persons_baseline.at[i,'Planned Mode Baseline'] = list(plans_baseline['legMode'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1])])[0]
    except:
        print('Warning', row, 'Baseline planned mode')
    try:
        persons_baseline.at[i,'Planned Mode TR'] = list(plans_TR['legMode'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1])])[0]
    except:
        print('Warning', row, 'TR planned mode')
    try:
        persons_baseline.at[i,'Chosen Mode Baseline'] = list(mode_choice_baseline['mode'][(mode_choice_baseline['person']==row[0])&(mode_choice_baseline['time']<=PtoPTss_baseline_dict[row][2]+1)])[-1]
                                                        
    except:
        print('Warning', row, 'mode_choice_baseline')
    try:
        persons_baseline.at[i,'Chosen Mode TR'] = list(mode_choice_TR['mode'][(mode_choice_TR['person']==row[0])&(mode_choice_TR['time']<=PtoPTss_TR_dict[row][2]+1)])[-1]
    except:
        print('Warning', row, 'mode_choice_TR')
###################################
for row in PtoPTss_TR_VN_dict.keys():
    i+=1
    if i%50==0:
        print(i)
        
        
    try:
        persons_baseline.at[i,'Person'] = row[0]
    except:
        print('Warning', row, 'person')
    try:
        persons_baseline.at[i,'Plan Index'] = int(row[1])
    except:
        print('Warning', row, 'plan_index')
    try:
        persons_baseline.at[i,'Activity From Baseline'] = list(plans_baseline['activityType'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]-1)])[0]
    except:
        print('Warning', row, 'Activity From Baseline')
    try:
        persons_baseline.at[i,'Activity To Baseline'] = list(plans_baseline['activityType'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]+1)])[0]
    except:
        print('Warning', row, 'Activity To Baseline')
    try:
        persons_baseline.at[i,'Activity From TR'] = list(plans_TR['activityType'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]-1)])[0]
    except:
        print('Warning', row, 'Activity From TR')
    try:
        persons_baseline.at[i,'Activity To TR'] = list(plans_TR['activityType'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]+1)])[0]
    except:
        print('Warning', row, 'Activity To TR')  
    try:
        persons_baseline.at[i,'Planned Depart Time Baseline'] = int(list(plans_baseline['activityEndTime'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]-1)])[0])
    except:
        print('Warning', row, 'plan_dep_time_baseline')
    try:
        persons_baseline.at[i,'Planned Depart Time TR'] = int(list(plans_TR['activityEndTime'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]-1)])[0])
    except:
        print('Warning', row, 'plan_dep_time_TR')
    try:
        persons_baseline.at[i,'Project Tried'] = 'SF - Van Ness'
    except:
        print('Warning', row, 'project tried')
    try:
        persons_baseline.at[i,'Potential INEXUS Baseline'] = np.mean(list(person_trips_baseline['logsum_trip_Potential_INEXUS'][(person_trips_baseline['person_id']==row[0])]))
    except:
        print('Warning', row, 'Potential INEXUS Baseline')
    try:
        persons_baseline.at[i,'Potential INEXUS TR'] = np.mean(list(person_trips_TR['logsum_trip_Potential_INEXUS'][(person_trips_TR['person_id']==row[0])]))
    except:
        print('Warning', row, 'Potential INEXUS TR')
    try:
        persons_baseline.at[i,'Destination LOG SUM X Baseline'] = np.mean(list(person_trips_baseline['destination_logsum_x'][(person_trips_baseline['person_id']==row[0])]))
    except:
        print('Warning', row, 'Destination LOG SUM X Baseline')
    try:
        persons_baseline.at[i,'Destination LOG SUM X TR'] = np.mean(list(person_trips_TR['destination_logsum_x'][(person_trips_TR['person_id']==row[0])]))
    except:
        print('Warning', row, 'Destination LOG SUM X TR')
    try:
        persons_baseline.at[i,'Destination LOG SUM Y Baseline'] = np.mean(list(person_trips_baseline['destination_logsum_y'][(person_trips_baseline['person_id']==row[0])]))
    except:
        print('Warning', row, 'Destination LOG SUM Y Baseline')
    try:
        persons_baseline.at[i,'Destination LOG SUM Y TR'] = np.mean(list(person_trips_TR['destination_logsum_y'][(person_trips_TR['person_id']==row[0])]))
    except:
        print('Warning', row, 'Destination LOG SUM Y TR')
    try:
        persons_baseline.at[i,'LOG SUM Tours Mode AS Baseline'] = np.mean(list(person_trips_baseline['logsum_tours_mode_AS_tours'][(person_trips_baseline['person_id']==row[0])]))
    except:
        print('Warning', row, 'LOG SUM Tours Mode AS Baseline')
    try:
        persons_baseline.at[i,'LOG SUM Tours Mode AS TR'] = np.mean(list(person_trips_TR['logsum_tours_mode_AS_tours'][(person_trips_TR['person_id']==row[0])]))
    except:
        print('Warning', row, 'LOG SUM Tours Mode AS TR')
    try:
        persons_baseline.at[i,'LOG SUM Work PLace Location Baseline'] = np.mean(list(person_trips_baseline['workplace_location_logsum'][(person_trips_baseline['person_id']==row[0])]))
    except:
        print('Warning', row, 'LOG SUM Work PLace Location Baseline')
    try:
        persons_baseline.at[i,'LOG SUM Work PLace Location TR'] = np.mean(list(person_trips_TR['workplace_location_logsum'][(person_trips_TR['person_id']==row[0])]))
    except:
        print('Warning', row, 'LOG SUM Work PLace Location TR') 
    try:
        persons_baseline.at[i,'Diff Potential INEXUS'] = persons_baseline.at[i,'Potential INEXUS TR']-persons_baseline.at[i,'Potential INEXUS Baseline']
    except:
        print('Warning', row, 'Diff Potential INEXUS')
    try:
        persons_baseline.at[i,'Diff Destination LOG SUM X'] = persons_baseline.at[i,'Destination LOG SUM X TR']-persons_baseline.at[i,'Destination LOG SUM X Baseline']
    except:
        print('Warning', row, 'Diff Destination LOG SUM X')
    try:
        persons_baseline.at[i,'Diff Destination LOG SUM Y'] = persons_baseline.at[i,'Destination LOG SUM Y TR']-persons_baseline.at[i,'Destination LOG SUM Y Baseline']
    except:
        print('Warning', row, 'Diff Destination LOG SUM Y')
    try:
        persons_baseline.at[i,'Diff LOG SUM Tours Mode AS'] = persons_baseline.at[i,'LOG SUM Tours Mode AS TR']-persons_baseline.at[i,'LOG SUM Tours Mode AS Baseline']
    except:
        print('Warning', row, 'Diff LOG SUM Tours Mode AS')
    try:
        persons_baseline.at[i,'Diff LOG SUM Work PLace Location'] = persons_baseline.at[i,'LOG SUM Work PLace Location TR']-persons_baseline.at[i,'LOG SUM Work PLace Location Baseline']
    except:
        print('Warning', row, 'Diff LOG SUM Work PLace Location')
    try:
        persons_baseline.at[i,'Vehicles Used Baseline'] = np.unique(list(PtoPTss_baseline['vehicleID'][(PtoPTss_baseline['personID']==row[0])&(PtoPTss_baseline['planIndex']==row[1])]))
    except:
        print('Warning', row, 'Vehicles Used Baseline')
    try:
        persons_baseline.at[i,'Vehicles Used TR'] = np.unique(list(PtoPTss_TR['vehicleID'][(PtoPTss_TR['personID']==row[0])&(PtoPTss_TR['planIndex']==row[1])]))
    except:
        print('Warning', row, 'Vehicles Used TR')
    try:
        persons_baseline.at[i,'Bus agencies Used Baseline'] = np.unique(list(PtoPTss_baseline_bus['vehicle2'][(PtoPTss_baseline_bus['personID']==row[0])&(PtoPTss_baseline_bus['planIndex']==row[1])]))
    except:
        print('Warning', row, 'Bus agencies Used Baseline')
    try:
        persons_baseline.at[i,'Bus agencies Used TR'] = np.unique(list(PtoPTss_TR_bus['vehicle2'][(PtoPTss_TR_bus['personID']==row[0])&(PtoPTss_TR_bus['planIndex']==row[1])]))
    except:
        print('Warning', row, 'Bus agencies Used TR')
    try:
        persons_baseline.at[i,'First Bus agency Used Baseline'] = np.unique(list(PtoPTss_baseline_bus['vehicle2'][(PtoPTss_baseline_bus['personID']==row[0])&(PtoPTss_baseline_bus['planIndex']==row[1])]))[0]
    except:
        print('Warning', row, 'First Bus agency Used Baseline')
    try:
        persons_baseline.at[i,'First Bus agency Used TR'] = np.unique(list(PtoPTss_TR_bus['vehicle2'][(PtoPTss_TR_bus['personID']==row[0])&(PtoPTss_TR_bus['planIndex']==row[1])]))[0]
    except:
        print('Warning', row, 'First Bus agency Used TR')  
    try:
        persons_baseline.at[i,'Trip Length TR'] = int(PtoPTss_TR_dict[row][0])
    except:
        print('Warning', row, 'TR_length')
    try:
        persons_baseline.at[i,'Trip Duration TR'] = int(PtoPTss_TR_dict[row][1])
    except:
        print('Warning', row, 'TR_duration')
    try:
        persons_baseline.at[i,'Trip Length Baseline'] = int(PtoPTss_baseline_dict[row][0])
    except:
        print('Warning', row, 'baseline_length')
    try:
        persons_baseline.at[i,'Trip Duration Baseline'] = int(PtoPTss_baseline_dict[row][1])
    except:
        print('Warning', row, 'baseline_duration')
    try:
        persons_baseline.at[i,'Diff Length'] = int(persons_baseline.at[i,'Trip Length TR']-persons_baseline.at[i,'Trip Length Baseline'])
    except:
        print('Warning', row, 'diff_length')
    try:
        persons_baseline.at[i,'Diff Duration'] = int(persons_baseline.at[i,'Trip Duration TR']-persons_baseline.at[i,'Trip Duration Baseline'])
    except:
        print('Warning', row, 'diff_duration')
    try:
        persons_baseline.at[i,'X Activity From TR'] = list(plans_TR['activityLocationX'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]-1)])[0]
    except:
        print('Warning', row, 'x_activity_from_TR')
    try:
        persons_baseline.at[i,'Y Activity From TR'] = list(plans_TR['activityLocationY'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]-1)])[0]
    except:
        print('Warning', row, 'y_activity_From_TR')
    try:
        persons_baseline.at[i,'X Activity To TR'] = list(plans_TR['activityLocationX'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]+1)])[0]
    except:
        print('Warning', row, 'x_activity_To_TR')
    try:
        persons_baseline.at[i,'Y Activity To TR'] = list(plans_TR['activityLocationY'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]+1)])[0]
    except:
        print('Warning', row, 'y_activity_To_TR')   
    try:
        persons_baseline.at[i,'X Activity From Baseline'] = list(plans_baseline['activityLocationX'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]-1)])[0]
    except:
        print('Warning', row, 'x_activity_from_Baseline')
    try:
        persons_baseline.at[i,'Y Activity From Baseline'] = list(plans_baseline['activityLocationY'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]-1)])[0]
    except:
        print('Warning', row, 'y_activity_From_Baseline')
    try:
        persons_baseline.at[i,'X Activity To Baseline'] = list(plans_baseline['activityLocationX'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]+1)])[0]
    except:
        print('Warning', row, 'x_activity_To_Baseline')
    try:
        persons_baseline.at[i,'Y Activity To Baseline'] = list(plans_baseline['activityLocationY'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]+1)])[0]
    except:
        print('Warning', row, 'y_activity_To_Baseline')   
    try:
        persons_baseline.at[i,'Vehicle Types Used Baseline'] = np.unique(list(PtoPTss_baseline['mode'][(PtoPTss_baseline['personID']==row[0])&(PtoPTss_baseline['planIndex']==row[1])]))
    except:
        print('Warning', row, 'Vehicle Types Used baseline')
    try:
        persons_baseline.at[i,'Vehicle Types Used TR'] = np.unique(list(PtoPTss_TR['mode'][(PtoPTss_TR['personID']==row[0])&(PtoPTss_TR['planIndex']==row[1])]))
    except:
        print('Warning', row, 'Vehicle Types Used TR')
    try:
        persons_baseline.at[i,'Planned Mode Baseline'] = list(plans_baseline['legMode'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1])])[0]
    except:
        print('Warning', row, 'Baseline planned mode')
    try:
        persons_baseline.at[i,'Planned Mode TR'] = list(plans_TR['legMode'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1])])[0]
    except:
        print('Warning', row, 'TR planned mode')
    try:
        persons_baseline.at[i,'Chosen Mode Baseline'] = list(mode_choice_baseline['mode'][(mode_choice_baseline['person']==row[0])&(mode_choice_baseline['time']<=PtoPTss_baseline_dict[row][2]+1)])[-1]
                                                        
    except:
        print('Warning', row, 'mode_choice_baseline')
    try:
        persons_baseline.at[i,'Chosen Mode TR'] = list(mode_choice_TR['mode'][(mode_choice_TR['person']==row[0])&(mode_choice_TR['time']<=PtoPTss_TR_dict[row][2]+1)])[-1]
    except:
        print('Warning', row, 'mode_choice_TR')
###################################
for row in PtoPTss_TR_CA_dict.keys():
    i+=1
    if i%50==0:
        print(i)
        
        
    try:
        persons_baseline.at[i,'Person'] = row[0]
    except:
        print('Warning', row, 'person')
    try:
        persons_baseline.at[i,'Plan Index'] = int(row[1])
    except:
        print('Warning', row, 'plan_index')
    try:
        persons_baseline.at[i,'Activity From Baseline'] = list(plans_baseline['activityType'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]-1)])[0]
    except:
        print('Warning', row, 'Activity From Baseline')
    try:
        persons_baseline.at[i,'Activity To Baseline'] = list(plans_baseline['activityType'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]+1)])[0]
    except:
        print('Warning', row, 'Activity To Baseline')
    try:
        persons_baseline.at[i,'Activity From TR'] = list(plans_TR['activityType'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]-1)])[0]
    except:
        print('Warning', row, 'Activity From TR')
    try:
        persons_baseline.at[i,'Activity To TR'] = list(plans_TR['activityType'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]+1)])[0]
    except:
        print('Warning', row, 'Activity To TR')  
    try:
        persons_baseline.at[i,'Planned Depart Time Baseline'] = int(list(plans_baseline['activityEndTime'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]-1)])[0])
    except:
        print('Warning', row, 'plan_dep_time_baseline')
    try:
        persons_baseline.at[i,'Planned Depart Time TR'] = int(list(plans_TR['activityEndTime'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]-1)])[0])
    except:
        print('Warning', row, 'plan_dep_time_TR')
    try:
        persons_baseline.at[i,'Project Tried'] = 'CA - Electrification Project'
    except:
        print('Warning', row, 'project tried')
    try:
        persons_baseline.at[i,'Potential INEXUS Baseline'] = np.mean(list(person_trips_baseline['logsum_trip_Potential_INEXUS'][(person_trips_baseline['person_id']==row[0])]))
    except:
        print('Warning', row, 'Potential INEXUS Baseline')
    try:
        persons_baseline.at[i,'Potential INEXUS TR'] = np.mean(list(person_trips_TR['logsum_trip_Potential_INEXUS'][(person_trips_TR['person_id']==row[0])]))
    except:
        print('Warning', row, 'Potential INEXUS TR')
    try:
        persons_baseline.at[i,'Destination LOG SUM X Baseline'] = np.mean(list(person_trips_baseline['destination_logsum_x'][(person_trips_baseline['person_id']==row[0])]))
    except:
        print('Warning', row, 'Destination LOG SUM X Baseline')
    try:
        persons_baseline.at[i,'Destination LOG SUM X TR'] = np.mean(list(person_trips_TR['destination_logsum_x'][(person_trips_TR['person_id']==row[0])]))
    except:
        print('Warning', row, 'Destination LOG SUM X TR')
    try:
        persons_baseline.at[i,'Destination LOG SUM Y Baseline'] = np.mean(list(person_trips_baseline['destination_logsum_y'][(person_trips_baseline['person_id']==row[0])]))
    except:
        print('Warning', row, 'Destination LOG SUM Y Baseline')
    try:
        persons_baseline.at[i,'Destination LOG SUM Y TR'] = np.mean(list(person_trips_TR['destination_logsum_y'][(person_trips_TR['person_id']==row[0])]))
    except:
        print('Warning', row, 'Destination LOG SUM Y TR')
    try:
        persons_baseline.at[i,'LOG SUM Tours Mode AS Baseline'] = np.mean(list(person_trips_baseline['logsum_tours_mode_AS_tours'][(person_trips_baseline['person_id']==row[0])]))
    except:
        print('Warning', row, 'LOG SUM Tours Mode AS Baseline')
    try:
        persons_baseline.at[i,'LOG SUM Tours Mode AS TR'] = np.mean(list(person_trips_TR['logsum_tours_mode_AS_tours'][(person_trips_TR['person_id']==row[0])]))
    except:
        print('Warning', row, 'LOG SUM Tours Mode AS TR')
    try:
        persons_baseline.at[i,'LOG SUM Work PLace Location Baseline'] = np.mean(list(person_trips_baseline['workplace_location_logsum'][(person_trips_baseline['person_id']==row[0])]))
    except:
        print('Warning', row, 'LOG SUM Work PLace Location Baseline')
    try:
        persons_baseline.at[i,'LOG SUM Work PLace Location TR'] = np.mean(list(person_trips_TR['workplace_location_logsum'][(person_trips_TR['person_id']==row[0])]))
    except:
        print('Warning', row, 'LOG SUM Work PLace Location TR') 
    try:
        persons_baseline.at[i,'Diff Potential INEXUS'] = persons_baseline.at[i,'Potential INEXUS TR']-persons_baseline.at[i,'Potential INEXUS Baseline']
    except:
        print('Warning', row, 'Diff Potential INEXUS')
    try:
        persons_baseline.at[i,'Diff Destination LOG SUM X'] = persons_baseline.at[i,'Destination LOG SUM X TR']-persons_baseline.at[i,'Destination LOG SUM X Baseline']
    except:
        print('Warning', row, 'Diff Destination LOG SUM X')
    try:
        persons_baseline.at[i,'Diff Destination LOG SUM Y'] = persons_baseline.at[i,'Destination LOG SUM Y TR']-persons_baseline.at[i,'Destination LOG SUM Y Baseline']
    except:
        print('Warning', row, 'Diff Destination LOG SUM Y')
    try:
        persons_baseline.at[i,'Diff LOG SUM Tours Mode AS'] = persons_baseline.at[i,'LOG SUM Tours Mode AS TR']-persons_baseline.at[i,'LOG SUM Tours Mode AS Baseline']
    except:
        print('Warning', row, 'Diff LOG SUM Tours Mode AS')
    try:
        persons_baseline.at[i,'Diff LOG SUM Work PLace Location'] = persons_baseline.at[i,'LOG SUM Work PLace Location TR']-persons_baseline.at[i,'LOG SUM Work PLace Location Baseline']
    except:
        print('Warning', row, 'Diff LOG SUM Work PLace Location')
    try:
        persons_baseline.at[i,'Vehicles Used Baseline'] = np.unique(list(PtoPTss_baseline['vehicleID'][(PtoPTss_baseline['personID']==row[0])&(PtoPTss_baseline['planIndex']==row[1])]))
    except:
        print('Warning', row, 'Vehicles Used Baseline')
    try:
        persons_baseline.at[i,'Vehicles Used TR'] = np.unique(list(PtoPTss_TR['vehicleID'][(PtoPTss_TR['personID']==row[0])&(PtoPTss_TR['planIndex']==row[1])]))
    except:
        print('Warning', row, 'Vehicles Used TR')
    try:
        persons_baseline.at[i,'Bus agencies Used Baseline'] = np.unique(list(PtoPTss_baseline_bus['vehicle2'][(PtoPTss_baseline_bus['personID']==row[0])&(PtoPTss_baseline_bus['planIndex']==row[1])]))
    except:
        print('Warning', row, 'Bus agencies Used Baseline')
    try:
        persons_baseline.at[i,'Bus agencies Used TR'] = np.unique(list(PtoPTss_TR_bus['vehicle2'][(PtoPTss_TR_bus['personID']==row[0])&(PtoPTss_TR_bus['planIndex']==row[1])]))
    except:
        print('Warning', row, 'Bus agencies Used TR')
    try:
        persons_baseline.at[i,'First Bus agency Used Baseline'] = np.unique(list(PtoPTss_baseline_bus['vehicle2'][(PtoPTss_baseline_bus['personID']==row[0])&(PtoPTss_baseline_bus['planIndex']==row[1])]))[0]
    except:
        print('Warning', row, 'First Bus agency Used Baseline')
    try:
        persons_baseline.at[i,'First Bus agency Used TR'] = np.unique(list(PtoPTss_TR_bus['vehicle2'][(PtoPTss_TR_bus['personID']==row[0])&(PtoPTss_TR_bus['planIndex']==row[1])]))[0]
    except:
        print('Warning', row, 'First Bus agency Used TR')  
    try:
        persons_baseline.at[i,'Trip Length TR'] = int(PtoPTss_TR_dict[row][0])
    except:
        print('Warning', row, 'TR_length')
    try:
        persons_baseline.at[i,'Trip Duration TR'] = int(PtoPTss_TR_dict[row][1])
    except:
        print('Warning', row, 'TR_duration')
    try:
        persons_baseline.at[i,'Trip Length Baseline'] = int(PtoPTss_baseline_dict[row][0])
    except:
        print('Warning', row, 'baseline_length')
    try:
        persons_baseline.at[i,'Trip Duration Baseline'] = int(PtoPTss_baseline_dict[row][1])
    except:
        print('Warning', row, 'baseline_duration')
    try:
        persons_baseline.at[i,'Diff Length'] = int(persons_baseline.at[i,'Trip Length TR']-persons_baseline.at[i,'Trip Length Baseline'])
    except:
        print('Warning', row, 'diff_length')
    try:
        persons_baseline.at[i,'Diff Duration'] = int(persons_baseline.at[i,'Trip Duration TR']-persons_baseline.at[i,'Trip Duration Baseline'])
    except:
        print('Warning', row, 'diff_duration')
    try:
        persons_baseline.at[i,'X Activity From TR'] = list(plans_TR['activityLocationX'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]-1)])[0]
    except:
        print('Warning', row, 'x_activity_from_TR')
    try:
        persons_baseline.at[i,'Y Activity From TR'] = list(plans_TR['activityLocationY'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]-1)])[0]
    except:
        print('Warning', row, 'y_activity_From_TR')
    try:
        persons_baseline.at[i,'X Activity To TR'] = list(plans_TR['activityLocationX'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]+1)])[0]
    except:
        print('Warning', row, 'x_activity_To_TR')
    try:
        persons_baseline.at[i,'Y Activity To TR'] = list(plans_TR['activityLocationY'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]+1)])[0]
    except:
        print('Warning', row, 'y_activity_To_TR')   
    try:
        persons_baseline.at[i,'X Activity From Baseline'] = list(plans_baseline['activityLocationX'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]-1)])[0]
    except:
        print('Warning', row, 'x_activity_from_Baseline')
    try:
        persons_baseline.at[i,'Y Activity From Baseline'] = list(plans_baseline['activityLocationY'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]-1)])[0]
    except:
        print('Warning', row, 'y_activity_From_Baseline')
    try:
        persons_baseline.at[i,'X Activity To Baseline'] = list(plans_baseline['activityLocationX'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]+1)])[0]
    except:
        print('Warning', row, 'x_activity_To_Baseline')
    try:
        persons_baseline.at[i,'Y Activity To Baseline'] = list(plans_baseline['activityLocationY'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]+1)])[0]
    except:
        print('Warning', row, 'y_activity_To_Baseline')   
    try:
        persons_baseline.at[i,'Vehicle Types Used Baseline'] = np.unique(list(PtoPTss_baseline['mode'][(PtoPTss_baseline['personID']==row[0])&(PtoPTss_baseline['planIndex']==row[1])]))
    except:
        print('Warning', row, 'Vehicle Types Used baseline')
    try:
        persons_baseline.at[i,'Vehicle Types Used TR'] = np.unique(list(PtoPTss_TR['mode'][(PtoPTss_TR['personID']==row[0])&(PtoPTss_TR['planIndex']==row[1])]))
    except:
        print('Warning', row, 'Vehicle Types Used TR')
    try:
        persons_baseline.at[i,'Planned Mode Baseline'] = list(plans_baseline['legMode'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1])])[0]
    except:
        print('Warning', row, 'Baseline planned mode')
    try:
        persons_baseline.at[i,'Planned Mode TR'] = list(plans_TR['legMode'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1])])[0]
    except:
        print('Warning', row, 'TR planned mode')
    try:
        persons_baseline.at[i,'Chosen Mode Baseline'] = list(mode_choice_baseline['mode'][(mode_choice_baseline['person']==row[0])&(mode_choice_baseline['time']<=PtoPTss_baseline_dict[row][2]+1)])[-1]
                                                        
    except:
        print('Warning', row, 'mode_choice_baseline')
    try:
        persons_baseline.at[i,'Chosen Mode TR'] = list(mode_choice_TR['mode'][(mode_choice_TR['person']==row[0])&(mode_choice_TR['time']<=PtoPTss_TR_dict[row][2]+1)])[-1]
    except:
        print('Warning', row, 'mode_choice_TR')
        
####################################
for row in PtoPTss_TR_AC_dict.keys():
    i+=1
    if i%50==0:
        print(i) 
        
        
    try:
        persons_baseline.at[i,'Person'] = row[0]
    except:
        print('Warning', row, 'person')
    try:
        persons_baseline.at[i,'Plan Index'] = int(row[1])
    except:
        print('Warning', row, 'plan_index')    
    try:
        persons_baseline.at[i,'Activity From Baseline'] = list(plans_baseline['activityType'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]-1)])[0]
    except:
        print('Warning', row, 'Activity From Baseline')
    try:
        persons_baseline.at[i,'Activity To Baseline'] = list(plans_baseline['activityType'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]+1)])[0]
    except:
        print('Warning', row, 'Activity To Baseline')
    try:
        persons_baseline.at[i,'Activity From TR'] = list(plans_TR['activityType'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]-1)])[0]
    except:
        print('Warning', row, 'Activity From TR')
    try:
        persons_baseline.at[i,'Activity To TR'] = list(plans_TR['activityType'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]+1)])[0]
    except:
        print('Warning', row, 'Activity To TR')   
    try:
        persons_baseline.at[i,'Planned Depart Time Baseline'] = int(list(plans_baseline['activityEndTime'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]-1)])[0])
    except:
        print('Warning', row, 'plan_dep_time_baseline')
    try:
        persons_baseline.at[i,'Planned Depart Time TR'] = int(list(plans_TR['activityEndTime'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]-1)])[0])
    except:
        print('Warning', row, 'plan_dep_time_TR')
    try:
        persons_baseline.at[i,'Project Tried'] = 'AC - 1TEMPO'
    except:
        print('Warning', row, 'project tried')
    try:
        persons_baseline.at[i,'Potential INEXUS Baseline'] = np.mean(list(person_trips_baseline['logsum_trip_Potential_INEXUS'][(person_trips_baseline['person_id']==row[0])]))
    except:
        print('Warning', row, 'Potential INEXUS Baseline')
    try:
        persons_baseline.at[i,'Potential INEXUS TR'] = np.mean(list(person_trips_TR['logsum_trip_Potential_INEXUS'][(person_trips_TR['person_id']==row[0])]))
    except:
        print('Warning', row, 'Potential INEXUS TR')
    try:
        persons_baseline.at[i,'Destination LOG SUM X Baseline'] = np.mean(list(person_trips_baseline['destination_logsum_x'][(person_trips_baseline['person_id']==row[0])]))
    except:
        print('Warning', row, 'Destination LOG SUM X Baseline')
    try:
        persons_baseline.at[i,'Destination LOG SUM X TR'] = np.mean(list(person_trips_TR['destination_logsum_x'][(person_trips_TR['person_id']==row[0])]))
    except:
        print('Warning', row, 'Destination LOG SUM X TR')
    try:
        persons_baseline.at[i,'Destination LOG SUM Y Baseline'] = np.mean(list(person_trips_baseline['destination_logsum_y'][(person_trips_baseline['person_id']==row[0])]))
    except:
        print('Warning', row, 'Destination LOG SUM Y Baseline')
    try:
        persons_baseline.at[i,'Destination LOG SUM Y TR'] = np.mean(list(person_trips_TR['destination_logsum_y'][(person_trips_TR['person_id']==row[0])]))
    except:
        print('Warning', row, 'Destination LOG SUM Y TR')
    try:
        persons_baseline.at[i,'LOG SUM Tours Mode AS Baseline'] = np.mean(list(person_trips_baseline['logsum_tours_mode_AS_tours'][(person_trips_baseline['person_id']==row[0])]))
    except:
        print('Warning', row, 'LOG SUM Tours Mode AS Baseline')
    try:
        persons_baseline.at[i,'LOG SUM Tours Mode AS TR'] = np.mean(list(person_trips_TR['logsum_tours_mode_AS_tours'][(person_trips_TR['person_id']==row[0])]))
    except:
        print('Warning', row, 'LOG SUM Tours Mode AS TR')
    try:
        persons_baseline.at[i,'LOG SUM Work PLace Location Baseline'] = np.mean(list(person_trips_baseline['workplace_location_logsum'][(person_trips_baseline['person_id']==row[0])]))
    except:
        print('Warning', row, 'LOG SUM Work PLace Location Baseline')
    try:
        persons_baseline.at[i,'LOG SUM Work PLace Location TR'] = np.mean(list(person_trips_TR['workplace_location_logsum'][(person_trips_TR['person_id']==row[0])]))
    except:
        print('Warning', row, 'LOG SUM Work PLace Location TR') 
    try:
        persons_baseline.at[i,'Diff Potential INEXUS'] = persons_baseline.at[i,'Potential INEXUS TR']-persons_baseline.at[i,'Potential INEXUS Baseline']
    except:
        print('Warning', row, 'Diff Potential INEXUS')
    try:
        persons_baseline.at[i,'Diff Destination LOG SUM X'] = persons_baseline.at[i,'Destination LOG SUM X TR']-persons_baseline.at[i,'Destination LOG SUM X Baseline']
    except:
        print('Warning', row, 'Diff Destination LOG SUM X')
    try:
        persons_baseline.at[i,'Diff Destination LOG SUM Y'] = persons_baseline.at[i,'Destination LOG SUM Y TR']-persons_baseline.at[i,'Destination LOG SUM Y Baseline']
    except:
        print('Warning', row, 'Diff Destination LOG SUM Y')
    try:
        persons_baseline.at[i,'Diff LOG SUM Tours Mode AS'] = persons_baseline.at[i,'LOG SUM Tours Mode AS TR']-persons_baseline.at[i,'LOG SUM Tours Mode AS Baseline']
    except:
        print('Warning', row, 'Diff LOG SUM Tours Mode AS')
    try:
        persons_baseline.at[i,'Diff LOG SUM Work PLace Location'] = persons_baseline.at[i,'LOG SUM Work PLace Location TR']-persons_baseline.at[i,'LOG SUM Work PLace Location Baseline']
    except:
        print('Warning', row, 'Diff LOG SUM Work PLace Location')
    try:
        persons_baseline.at[i,'Vehicles Used Baseline'] = np.unique(list(PtoPTss_baseline['vehicleID'][(PtoPTss_baseline['personID']==row[0])&(PtoPTss_baseline['planIndex']==row[1])]))
    except:
        print('Warning', row, 'Vehicles Used Baseline')
    try:
        persons_baseline.at[i,'Vehicles Used TR'] = np.unique(list(PtoPTss_TR['vehicleID'][(PtoPTss_TR['personID']==row[0])&(PtoPTss_TR['planIndex']==row[1])]))
    except:
        print('Warning', row, 'Vehicles Used TR')    
    try:
        persons_baseline.at[i,'Bus agencies Used Baseline'] = np.unique(list(PtoPTss_baseline_bus['vehicle2'][(PtoPTss_baseline_bus['personID']==row[0])&(PtoPTss_baseline_bus['planIndex']==row[1])]))
    except:
        print('Warning', row, 'Bus agencies Used Baseline')
    try:
        persons_baseline.at[i,'Bus agencies Used TR'] = np.unique(list(PtoPTss_TR_bus['vehicle2'][(PtoPTss_TR_bus['personID']==row[0])&(PtoPTss_TR_bus['planIndex']==row[1])]))
    except:
        print('Warning', row, 'Bus agencies Used TR')
    try:
        persons_baseline.at[i,'First Bus agency Used Baseline'] = np.unique(list(PtoPTss_baseline_bus['vehicle2'][(PtoPTss_baseline_bus['personID']==row[0])&(PtoPTss_baseline_bus['planIndex']==row[1])]))[0]
    except:
        print('Warning', row, 'First Bus agency Used Baseline')
    try:
        persons_baseline.at[i,'First Bus agency Used TR'] = np.unique(list(PtoPTss_TR_bus['vehicle2'][(PtoPTss_TR_bus['personID']==row[0])&(PtoPTss_TR_bus['planIndex']==row[1])]))[0]
    except:
        print('Warning', row, 'First Bus agency Used TR')
    try:
        persons_baseline.at[i,'Trip Length TR'] = int(PtoPTss_TR_dict[row][0])
    except:
        print('Warning', row, 'TR_length')
    try:
        persons_baseline.at[i,'Trip Duration TR'] = int(PtoPTss_TR_dict[row][1])
    except:
        print('Warning', row, 'TR_duration')
    try:
        persons_baseline.at[i,'Trip Length Baseline'] = int(PtoPTss_baseline_dict[row][0])
    except:
        print('Warning', row, 'baseline_length')
    try:
        persons_baseline.at[i,'Trip Duration Baseline'] = int(PtoPTss_baseline_dict[row][1])
    except:
        print('Warning', row, 'baseline_duration')
    try:
        persons_baseline.at[i,'Diff Length'] = int(persons_baseline.at[i,'Trip Length TR']-persons_baseline.at[i,'Trip Length Baseline'])
    except:
        print('Warning', row, 'diff_length')
    try:
        persons_baseline.at[i,'Diff Duration'] = int(persons_baseline.at[i,'Trip Duration TR']-persons_baseline.at[i,'Trip Duration Baseline'])
    except:
        print('Warning', row, 'diff_duration')
    try:
        persons_baseline.at[i,'X Activity From TR'] = list(plans_TR['activityLocationX'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]-1)])[0]
    except:
        print('Warning', row, 'x_activity_from_TR')
    try:
        persons_baseline.at[i,'Y Activity From TR'] = list(plans_TR['activityLocationY'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]-1)])[0]
    except:
        print('Warning', row, 'y_activity_From_TR')
    try:
        persons_baseline.at[i,'X Activity To TR'] = list(plans_TR['activityLocationX'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]+1)])[0]
    except:
        print('Warning', row, 'x_activity_To_TR')
    try:
        persons_baseline.at[i,'Y Activity To TR'] = list(plans_TR['activityLocationY'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]+1)])[0]
    except:
        print('Warning', row, 'y_activity_To_TR')   
    try:
        persons_baseline.at[i,'X Activity From Baseline'] = list(plans_baseline['activityLocationX'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]-1)])[0]
    except:
        print('Warning', row, 'x_activity_from_Baseline')
    try:
        persons_baseline.at[i,'Y Activity From Baseline'] = list(plans_baseline['activityLocationY'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]-1)])[0]
    except:
        print('Warning', row, 'y_activity_From_Baseline')
    try:
        persons_baseline.at[i,'X Activity To Baseline'] = list(plans_baseline['activityLocationX'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]+1)])[0]
    except:
        print('Warning', row, 'x_activity_To_Baseline')
    try:
        persons_baseline.at[i,'Y Activity To Baseline'] = list(plans_baseline['activityLocationY'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]+1)])[0]
    except:
        print('Warning', row, 'y_activity_To_Baseline')   
    try:
        persons_baseline.at[i,'Vehicle Types Used Baseline'] = np.unique(list(PtoPTss_baseline['mode'][(PtoPTss_baseline['personID']==row[0])&(PtoPTss_baseline['planIndex']==row[1])]))
    except:
        print('Warning', row, 'Vehicle Types Used baseline')
    try:
        persons_baseline.at[i,'Vehicle Types Used TR'] = np.unique(list(PtoPTss_TR['mode'][(PtoPTss_TR['personID']==row[0])&(PtoPTss_TR['planIndex']==row[1])]))
    except:
        print('Warning', row, 'Vehicle Types Used TR')
    try:
        persons_baseline.at[i,'Planned Mode Baseline'] = list(plans_baseline['legMode'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1])])[0]
    except:
        print('Warning', row, 'Baseline planned mode')
    try:
        persons_baseline.at[i,'Planned Mode TR'] = list(plans_TR['legMode'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1])])[0]
    except:
        print('Warning', row, 'TR planned mode')
    try:
        plan_dep_time_baseline = list(plans_baseline['activityEndTime'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1]-1)])[0]
        persons_baseline.at[i,'Chosen Mode Baseline'] = list(mode_choice_baseline['mode'][(mode_choice_baseline['person']==row[0])&(mode_choice_baseline['time']<=plan_dep_time_baseline)])[-1]
    except:
        print('Warning', row, 'mode_choice_baseline')
    try:
        persons_baseline.at[i,'Chosen Mode TR'] = list(mode_choice_TR['mode'][(mode_choice_TR['person']==row[0])&(mode_choice_TR['time']<=persons_baseline.at[i,'Planned Depart Time TR'])])[-1]
    except:
        print('Warning', row, 'mode_choice_TR')

    try:
        persons_baseline.at[i,'Chosen Mode Baseline'] = list(mode_choice_baseline['mode'][(mode_choice_baseline['person']==row[0])&(mode_choice_baseline['time']<=PtoPTss_baseline_dict[row][2]+1)])[-1]
                                                        
    except:
        print('Warning', row, 'mode_choice_baseline')
    try:
        persons_baseline.at[i,'Chosen Mode TR'] = list(mode_choice_TR['mode'][(mode_choice_TR['person']==row[0])&(mode_choice_TR['time']<=PtoPTss_TR_dict[row][2]+1)])[-1]
    except:
        print('Warning', row, 'mode_choice_TR')

        

        
        
        

        
        


1777
4427
2413
2728
Warning (13884, 7) First Bus agency Used Baseline
Warning (51178, 9) First Bus agency Used Baseline
Warning (145821, 3) First Bus agency Used Baseline
50
Warning (451810, 11) First Bus agency Used Baseline
Warning (451810, 11) baseline_length
Warning (451810, 11) baseline_duration
Warning (451810, 11) diff_length
Warning (451810, 11) diff_duration
Warning (451810, 11) mode_choice_baseline


/Users/cpoliziani/opt/miniconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/cpoliziani/opt/miniconda3/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/cpoliziani/opt/miniconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/cpoliziani/opt/miniconda3/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Warning (715308, 17) First Bus agency Used Baseline
100
Warning (736893, 3) First Bus agency Used Baseline
Warning (748300, 7) First Bus agency Used Baseline
Warning (877813, 5) First Bus agency Used Baseline
Warning (877813, 7) First Bus agency Used Baseline
150
Warning (1251735, 5) First Bus agency Used Baseline


/Users/cpoliziani/opt/miniconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/cpoliziani/opt/miniconda3/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Warning (1691866, 9) First Bus agency Used Baseline
Warning (1702037, 7) First Bus agency Used Baseline
Warning (1702037, 9) First Bus agency Used Baseline
Warning (1725867, 9) First Bus agency Used Baseline
Warning (1816554, 1) First Bus agency Used Baseline
Warning (1816554, 3) First Bus agency Used Baseline
Warning (1859369, 1) First Bus agency Used Baseline
Warning (1944208, 3) First Bus agency Used Baseline
Warning (2062506, 7) First Bus agency Used Baseline
200
Warning (2103628, 3) First Bus agency Used Baseline
Warning (2103628, 3) baseline_length
Warning (2103628, 3) baseline_duration
Warning (2103628, 3) diff_length
Warning (2103628, 3) diff_duration
Warning (2103628, 3) mode_choice_baseline
Warning (2161752, 19) First Bus agency Used Baseline
Warning (2161752, 19) baseline_length
Warning (2161752, 19) baseline_duration
Warning (2161752, 19) diff_length
Warning (2161752, 19) diff_duration
Warning (2161752, 19) mode_choice_baseline
Warning (2161752, 27) First Bus agency Used Ba

...

Warning (892095, 1) mode_choice_baseline
Warning (910780, 1) mode_choice_baseline
Warning (916429, 1) mode_choice_baseline
Warning (940623, 1) mode_choice_baseline
Warning (940623, 1) mode_choice_TR
Warning (1057141, 1) mode_choice_baseline
Warning (1057141, 1) mode_choice_TR
11100
Warning (1058786, 1) mode_choice_baseline
Warning (1241028, 1) First Bus agency Used Baseline
Warning (1241028, 1) mode_choice_baseline
Warning (1293851, 13) First Bus agency Used Baseline
Warning (1310694, 1) mode_choice_TR
Warning (1370233, 5) First Bus agency Used Baseline
Warning (1441924, 1) mode_choice_baseline
Warning (1450732, 3) First Bus agency Used Baseline
Warning (1527884, 3) First Bus agency Used Baseline
Warning (1540643, 1) mode_choice_baseline
Warning (1547459, 1) mode_choice_baseline
Warning (1599341, 1) mode_choice_TR
Warning (1618230, 1) mode_choice_baseline
Warning (1685658, 3) First Bus agency Used Baseline
Warning (1698079, 11) First Bus agency Used Baseline
Warning (1699486, 1) mode_c

/Users/cpoliziani/opt/miniconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/cpoliziani/opt/miniconda3/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Warning (1825507, 1) First Bus agency Used Baseline
Warning (1825507, 1) mode_choice_baseline
Warning (1825507, 3) First Bus agency Used Baseline
Warning (1910889, 1) First Bus agency Used Baseline
Warning (1910889, 1) mode_choice_TR
Warning (1991871, 1) mode_choice_baseline
Warning (1991871, 1) mode_choice_TR
Warning (2017575, 5) First Bus agency Used Baseline
Warning (2041280, 1) mode_choice_baseline
Warning (2089106, 1) mode_choice_baseline
Warning (2096404, 1) mode_choice_baseline
Warning (2096404, 1) mode_choice_TR
Warning (2096404, 3) First Bus agency Used Baseline
Warning (2120911, 3) First Bus agency Used Baseline
Warning (2126645, 1) mode_choice_TR
11200
Warning (2332511, 5) First Bus agency Used Baseline
Warning (2365402, 7) First Bus agency Used Baseline
Warning (2393203, 1) First Bus agency Used Baseline
Warning (2393203, 1) mode_choice_baseline
Warning (2393203, 1) mode_choice_TR
Warning (2393203, 3) First Bus agency Used Baseline
Warning (2405007, 1) First Bus agency Used

/Users/cpoliziani/opt/miniconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/cpoliziani/opt/miniconda3/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Warning (6201952, 1) First Bus agency Used Baseline
Warning (6201952, 1) mode_choice_baseline
Warning (6232353, 5) First Bus agency Used Baseline


In [13]:
zipcode = gpd.read_file('/Users/cpoliziani/Downloads/TAZ to ZIP/ZCTA2010.shp')
persons_baseline = addGeometryIdToDataFrame(persons_baseline, zipcode, 'X Activity From TR', 'Y Activity From TR', 'ZIP Departure TR')
persons_baseline = addGeometryIdToDataFrame(persons_baseline, zipcode, 'X Activity To TR', 'Y Activity To TR', 'ZIP Arrival TR')
persons_baseline = addGeometryIdToDataFrame(persons_baseline, zipcode, 'X Activity From Baseline', 'Y Activity From Baseline', 'ZIP Departure Baseline')
persons_baseline = addGeometryIdToDataFrame(persons_baseline, zipcode, 'X Activity To Baseline', 'Y Activity To Baseline', 'ZIP Arrival Baseline')


#Add Switch From Column SF - Van Ness
switch_type = []
for agencies_baseline, project in zip(persons_baseline['Bus agencies Used Baseline'],persons_baseline['Project Tried']):
    if len(agencies_baseline) == 0:
        switch_type.append('Switch from another mode')
    elif project == 'SF - Central Subway' and 'SF' in agencies_baseline:
        switch_type.append('Switch from same transit agency')
    elif project == 'AC - 1TEMPO' and 'AC' in agencies_baseline:
        switch_type.append('Switch from same transit agency')
    elif project == 'CA - Electrification Project' and 'ca' in agencies_baseline:
        switch_type.append('Switch from same transit agency')
    elif project == 'SF - Van Ness' and 'SF' in agencies_baseline:
        switch_type.append('Switch from same transit agency')
    else:
        switch_type.append('Switch from another transit agency')

persons_baseline['Switch From'] = switch_type

persons_baseline.to_csv('/Users/cpoliziani/Downloads/person_database2018it5_3.csv')


In [ ]:
persons_baseline

In [ ]:
# READ AND CORRECT LINK TABLE RESULTS
link_table_5fleets = pd.read_csv(dataFilepath_sim[1][:-26]+'postprocessOutputs/'+dataFilepath_sim[1].split('/')[6]+'/linkTable.csv.gz') # to Update
link_table_bs = pd.read_csv(dataFilepath_sim[0][:-26]+'postprocessOutputs/'+dataFilepath_sim[0].split('/')[6]+'/linkTable.csv.gz') # to Update
    
link_table_bs = link_table_bs.fillna(0)
bgids = []
wrong = 0
for bg in link_table_bs['bgid']:
    bgids.append('0'+str(int(bg)))
    if bg ==0:
        wrong +=1
print('wrong',wrong)
link_table_bs['bgid'] = bgids

link_table_5fleets = link_table_5fleets.fillna(0)
bgids = []
wrong = 0
for bg in link_table_5fleets['bgid']:
    bgids.append('0'+str(int(bg)))
    if bg ==0:
        wrong +=1
print('wrong',wrong)
link_table_5fleets['bgid'] = bgids

#Filter volume higher than 5
link_table_bs_volume = link_table_bs[link_table_bs['volume']>5]
link_table_5fleets_volume = link_table_5fleets[link_table_5fleets['volume']>5]

# AGGREGATE SPEEDS BY BLOCK ID
link_table_bs_grouped = link_table_bs_volume.groupby(['bgid']).apply(lambda x: np.mean(list(x.length/x.traveltime)))
link_table_5fleets_grouped = link_table_5fleets_volume.groupby(['bgid']).apply(lambda x: np.mean(list(x.length/x.traveltime)))
delta_speeds = {}
for bgid in link_table_bs_grouped.keys():
    try:
        delta_speeds[bgid] = (link_table_5fleets_grouped[bgid] - link_table_bs_grouped[bgid])/link_table_bs_grouped[bgid]
    except:
        print(bgid)
speeds_diff = pd.DataFrame()
speeds_diff['speeds_diff'] = delta_speeds.values()
speeds_diff['bgid'] = delta_speeds.keys()
speeds_diff.to_csv('/Users/cpoliziani/Downloads/diff_speeds_bgid.csv')



In [ ]:
link_table_5fleets = pd.read_csv(dataFilepath_sim[1][:-13]+'plans.csv.gz') # to Update
link_table_bs = pd.read_csv(dataFilepath_sim[0][:-13]+'plans.csv.gz') # to Update

In [ ]:
BGs = gpd.read_file('/vsicurl/https://github.com/LBNL-UCB-STI/beam-core-analysis/raw/main/Users/Zach/scenario/sfbay-blockgroups-2010/641aa0d4-ce5b-4a81-9c30-8790c4ab8cfb202047-1-wkkklf.j5ouj.shp')
link_table_bs = link_table_bs[link_table_bs['planSelected']==True]
link_table_5fleets = link_table_5fleets[link_table_5fleets['planSelected']==True]

link_table_5fleets = addGeometryIdToDataFrame(link_table_5fleets, BGs, 'activityLocationX', 'activityLocationY', 'bgid')
link_table_bs = addGeometryIdToDataFrame(link_table_bs, BGs, 'activityLocationX', 'activityLocationY', 'bgid')
    
# bgids = []
# wrong = 0
# for bg in link_table_bs['bgid']:
#     bgids.append('0'+str(int(bg)))
#     if bg ==0:
#         wrong +=1
# print('wrong',wrong)
# link_table_bs['bgid'] = bgids

# bgids = []
# wrong = 0
# for bg in link_table_5fleets['bgid']:
#     bgids.append('0'+str(int(bg)))
#     if bg ==0:
#         wrong +=1
# print('wrong',wrong)
# link_table_5fleets['bgid'] = bgids

#Filter volume higher than 5
# link_table_bs_volume = link_table_bs[link_table_bs['volume']>5]
# link_table_5fleets_volume = link_table_5fleets[link_table_5fleets['volume']>5]

# AGGREGATE SPEEDS BY BLOCK ID






In [ ]:


link_table_bs = link_table_bs.fillna(0)
link_table_5fleets = link_table_5fleets.fillna(0)


ink_table_bs_grouped = link_table_bs.groupby(['bgid']).apply(lambda x: len(x.activityType))
# link_table_5fleets_grouped = link_table_5fleets.groupby(['bgid']).apply(lambda x: x.activityType.value_counts())
delta_speeds = {}


In [ ]:
for bgid in link_table_bs_grouped.keys():
    try:
        delta_speeds[bgid] = (link_table_5fleets_grouped[bgid] - link_table_bs_grouped[bgid])/link_table_bs_grouped[bgid]
    except:
        print(bgid)
speeds_diff = pd.DataFrame()
speeds_diff['speeds_diff'] = delta_speeds.values()
speeds_diff['bgid'] = delta_speeds.keys()
speeds_diff.to_csv('/Users/cpoliziani/Downloads/diff_speeds_bgid.csv')

In [ ]:
ink_table_bs_grouped

In [ ]:
ink_table_bs_grouped

In [ ]:

# for row in PtoPTss_TR_CA_dict.keys():
#     i+=1
#     if i%1000==0:
#         print(i)
#     try:
#         persons_baseline.at[i,'person'] = row[0]
#         persons_baseline.at[i,'plan_index'] = row[1]
#         persons_baseline.at[i,'project tried'] = 'CA - Electrification Project'
#         persons_baseline.at[i,'TR_length'] = PtoPTss_TR_dict[row][0]
#         persons_baseline.at[i,'TR_duration'] = PtoPTss_TR_dict[row][1]
#         persons_baseline.at[i,'baseline_length'] = PtoPTss_baseline_dict[row][0]
#         persons_baseline.at[i,'baseline_duration'] = PtoPTss_baseline_dict[row][1]
#         persons_baseline.at[i,'diff_length'] = persons_baseline.at[i,'TR_length']-persons_baseline.at[i,'baseline_length']
#         persons_baseline.at[i,'diff_duration'] = persons_baseline.at[i,'TR_duration']-persons_baseline.at[i,'baseline_duration']
#         persons_baseline.at[i,'x_activity_TR'] = list(plans_TR['activityLocationX'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]-1)])[0]
#         persons_baseline.at[i,'y_activity_TR'] = list(plans_TR['activityLocationY'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]-1)])[0]
#         persons_baseline.at[i,'mode_baseline'] = list(plans_baseline['legMode'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1])])[0]
#     except:
#         persons_baseline.at[i,'x_activity_TR'] = list(plans_TR['activityLocationX'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]-1)])[0]
#         persons_baseline.at[i,'y_activity_TR'] = list(plans_TR['activityLocationY'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]-1)])[0]
#         persons_baseline.at[i,'mode_baseline'] = list(plans_baseline['legMode'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1])])[0]
#         persons_baseline.at[i,'person'] = row[0]
#         persons_baseline.at[i,'plan_index'] = row[1]
#         persons_baseline.at[i,'project tried'] = 'CA - Electrification Project'
#         print('WARNING:',row[0], row[1]) 


# for row in PtoPTss_TR_AC_dict.keys():
#     i+=1
#     if i%1000==0:
#         print(i)
#     try:
#         persons_baseline.at[i,'person'] = row[0]
#         persons_baseline.at[i,'plan_index'] = row[1]
#         persons_baseline.at[i,'project tried'] = 'AC - 1 Tempo BRT Line'
#         persons_baseline.at[i,'TR_length'] = PtoPTss_TR_dict[row][0]
#         persons_baseline.at[i,'TR_duration'] = PtoPTss_TR_dict[row][1]
#         persons_baseline.at[i,'baseline_length'] = PtoPTss_baseline_dict[row][0]
#         persons_baseline.at[i,'baseline_duration'] = PtoPTss_baseline_dict[row][1]
#         persons_baseline.at[i,'diff_length'] = persons_baseline.at[i,'TR_length']-persons_baseline.at[i,'baseline_length']
#         persons_baseline.at[i,'diff_duration'] = persons_baseline.at[i,'TR_duration']-persons_baseline.at[i,'baseline_duration']
#         persons_baseline.at[i,'x_activity_TR'] = list(plans_TR['activityLocationX'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]-1)])[0]
#         persons_baseline.at[i,'y_activity_TR'] = list(plans_TR['activityLocationY'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]-1)])[0]
#         persons_baseline.at[i,'mode_baseline'] = list(plans_baseline['legMode'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1])])[0]
#     except:
#         persons_baseline.at[i,'x_activity_TR'] = list(plans_TR['activityLocationX'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]-1)])[0]
#         persons_baseline.at[i,'y_activity_TR'] = list(plans_TR['activityLocationY'][(plans_TR['personId']==row[0])&(plans_TR['planElementIndex']==row[1]-1)])[0]
#         persons_baseline.at[i,'mode_baseline'] = list(plans_baseline['legMode'][(plans_baseline['personId']==row[0])&(plans_baseline['planElementIndex']==row[1])])[0]
#         persons_baseline.at[i,'person'] = row[0]
#         persons_baseline.at[i,'plan_index'] = row[1]
#         persons_baseline.at[i,'project tried'] = 'AC - 1 Tempo BRT Line'
#         print('WARNING:',row[0], row[1])




##############
# trips_TR = pd.concat([line_CA_trips_TR['trip_id'],line_CS_trips_TR['trip_id'],line_1T_trips_TR['trip_id']])
# for vehicle, person, plan_index, vehicle2 in zip(PtoPTss['vehicleID'],PtoPTss['personID'],PtoPTss['planIndex'],PtoPTss['vehicle2'] ):
#     if int(vehicle.split(':')[1]) in list(trips_TR):
#         i+=1
#         if i%1000==0:
#             print(i)
#         try:

#             persons_baseline.at[i,'person'] = person
#             persons_baseline.at[i,'plan_index'] = plan_index
#             persons_baseline.at[i,'mode_baseline'] = list(plans_baseline['legMode'][(plans_baseline['personId']==person)&(plans_baseline['planElementIndex']==plan_index)])[0]
#             persons_baseline.at[i,'x_activity_TR'] = list(plans_TR['activityLocationX'][(plans_TR['personId']==person)&(plans_TR['planElementIndex']==plan_index-1)])[0]
#             persons_baseline.at[i,'y_activity_TR'] = list(plans_TR['activityLocationY'][(plans_TR['personId']==person)&(plans_TR['planElementIndex']==plan_index-1)])[0]
#             persons_baseline.at[i,'vehicle_now'] = vehicle
#             if vehicle2 == 'SF':
#                 persons_baseline.at[i,'project'] = 'CS'
#             if vehicle2 == 'Ca':
#                 persons_baseline.at[i,'project'] = 'CA'
#             if vehicle2 == 'AC':
#                 persons_baseline.at[i,'project'] = '1T'

#             persons_baseline.at[i,'vehicle_baseline'] = list(PtoPTss_baseline['vehicleID'][(PtoPTss_baseline['personID']==person)&(PtoPTss_baseline['planIndex']==plan_index)])[0]
#             persons_baseline.at[i,'agency_baseline'] = list(PtoPTss_baseline['vehicle2'][(PtoPTss_baseline['personID']==person)&(PtoPTss_baseline['planIndex']==plan_index)])[0]
#             persons_baseline.at[i,'length_baseline'] = list(PtoPTss_baseline['length'][(PtoPTss_baseline['personID']==person)&(PtoPTss_baseline['planIndex']==plan_index)])[0]
#             persons_baseline.at[i,'length_TR'] = list(PtoPTss_TR['length'][(PtoPTss_baseline['personID']==person)&(PtoPTss_baseline['planIndex']==plan_index)])[0]
#             persons_baseline.at[i,'duration_baseline'] = list(PtoPTss_baseline['duration'][(PtoPTss_baseline['personID']==person)&(PtoPTss_baseline['planIndex']==plan_index)])[0]
#             persons_baseline.at[i,'duration_TR'] = list(PtoPTss_TR['duration'][(PtoPTss_baseline['personID']==person)&(PtoPTss_baseline['planIndex']==plan_index)])[0]
#             persons_baseline.at[i,'speed_baseline'] = persons_baseline.loc[i,'length_baseline']/persons_baseline.loc[i,'duration_baseline']
#             persons_baseline.at[i,'speed_TR'] = persons_baseline.loc[i,'length_TR']/persons_baseline.loc[i,'duration_TR']
#             persons_baseline.at[i,'diff_length'] = persons_baseline.loc[i,'length_TR']-persons_baseline.loc[i,'length_baseline'] 
#             persons_baseline.at[i,'diff_duration'] = persons_baseline.loc[i,'duration_TR']-persons_baseline.loc[i,'duration_baseline'] 
#             persons_baseline.at[i,'diff_speed'] = persons_baseline.loc[i,'speed_TR']-persons_baseline.loc[i,'speed_baseline'] 
#         except:
#             persons_baseline.at[i,'person'] = person
#             persons_baseline.at[i,'plan_index'] = plan_index
#             persons_baseline.at[i,'vehicle_now'] = vehicle
#             if vehicle2 == 'SF':
#                 persons_baseline.at[i,'project'] = 'CS'
#             if vehicle2 == 'Ca':
#                 persons_baseline.at[i,'project'] = 'CA'
#             if vehicle2 == 'AC':
#                 persons_baseline.at[i,'project'] = '1T'

#             print('PtoPTss_baseline not found person and plan index',person,plan_index, vehicle)
#     print('Warning')




# persons_baseline = pd.DataFrame(['person','plan_index','mode_baseline','vehicle_baseline','agency_baseline',
#                                  'x_activity_TR','y_activity_TR','vehicle_now'])
# i = 0
# PtoPTss=pd.concat(PtoPTss_AC_TR,PtoPTss_SF_TR,PtoPTss_CA_TR)
# for vehicle, person, plan_index in zip(PtoPTss['vehicleID'],PtoPTss['personID'],PtoPTss['planIndex'] ):
#     if int(vehicle.split(':')[1]) in list(line_1T_trips_TR):
#         i+=1
#         persons_baseline.loc[i,'person'] = person
#         persons_baseline.loc[i,'plan_index'] = plan_index
#         persons_baseline.loc[i,'mode_baseline'] = list(plans_baseline['legMode'][(plans_baseline['personId']==person)&(plans_baseline['planElementIndex']==plan_index)])[0]
#         persons_baseline.loc[i,'x_activity_TR'] = list(plans_TR['activityLocationX'][(plans_TR['personId']==person)&(plans_TR['planElementIndex']==plan_index-1)])[0]
#         persons_baseline.loc[i,'y_activity_TR'] = list(plans_TR['activityLocationY'][(plans_TR['personId']==person)&(plans_TR['planElementIndex']==plan_index-1)])[0]
#         persons_baseline.loc[i,'vehicle_now'] = '1T'
#         try:
#             persons_baseline.loc[i,'vehicle_baseline'] = list(PtoPTss_baseline['vehicleID'][(PtoPTss_baseline['personID']==person)&(PtoPTss_baseline['planIndex']==plan_index)])[0]
#             persons_baseline.loc[i,'agency_baseline'] = list(PtoPTss_baseline['vehicle2'][(PtoPTss_baseline['personID']==person)&(PtoPTss_baseline['planIndex']==plan_index)])[0]
#             persons_baseline.loc[i,'length_baseline'] = list(PtoPTss_baseline['length'][(PtoPTss_baseline['personID']==person)&(PtoPTss_baseline['planIndex']==plan_index)])[0]
#             persons_baseline.loc[i,'length_TR'] = list(PtoPTss_TR['length'][(PtoPTss_baseline['personID']==person)&(PtoPTss_baseline['planIndex']==plan_index)])[0]
#             persons_baseline.loc[i,'duration_baseline'] = list(PtoPTss_baseline['duration'][(PtoPTss_baseline['personID']==person)&(PtoPTss_baseline['planIndex']==plan_index)])[0]
#             persons_baseline.loc[i,'duration_TR'] = list(PtoPTss_TR['duration'][(PtoPTss_baseline['personID']==person)&(PtoPTss_baseline['planIndex']==plan_index)])[0]
#             persons_baseline.loc[i,'speed_baseline'] = persons_baseline.loc[i,'length_baseline']/persons_baseline.loc[i,'duration_baseline']
#             persons_baseline.loc[i,'speed_TR'] = persons_baseline.loc[i,'length_TR']/persons_baseline.loc[i,'duration_TR']
#             persons_baseline.loc[i,'diff_length'] = persons_baseline.loc[i,'length_TR']-persons_baseline.loc[i,'length_baseline'] 
#             persons_baseline.loc[i,'diff_duration'] = persons_baseline.loc[i,'duration_TR']-persons_baseline.loc[i,'duration_baseline'] 
#             persons_baseline.loc[i,'speed_length'] = persons_baseline.loc[i,'speed_TR']-persons_baseline.loc[i,'speed_baseline'] 
#         except:
#             persons_baseline.loc[i,'vehicle_baseline'] = None
#             persons_baseline.loc[i,'agency_baseline'] = None
#             print('PtoPTss_baseline not found person and plan index',person,plan_index, vehicle)


# persons_baseline.to_csv('/Users/cpoliziani/Downloads/person_database.csv')

# PtoPTss_baseline


# persons_baseline = pd.DataFrame()
# i = 0
# PtoPTss=pd.concat([PtoPTss_AC_TR,PtoPTss_SF_TR,PtoPTss_CA_TR])
# for vehicle, person, plan_index, vehicle2 in zip(PtoPTss['vehicleID'],PtoPTss['personID'],PtoPTss['planIndex'],PtoPTss['vehicle2'] ):
#     if int(vehicle.split(':')[1]) in list(line_1T_trips_TR):
#         i+=1

#             persons_baseline.at[i,'person'] = person
#             persons_baseline.at[i,'plan_index'] = plan_index
#             persons_baseline.at[i,'mode_baseline'] = list(plans_baseline['legMode'][(plans_baseline['personId']==person)&(plans_baseline['planElementIndex']==plan_index)])[0]
#             persons_baseline.at[i,'x_activity_TR'] = list(plans_TR['activityLocationX'][(plans_TR['personId']==person)&(plans_TR['planElementIndex']==plan_index-1)])[0]
#             persons_baseline.at[i,'y_activity_TR'] = list(plans_TR['activityLocationY'][(plans_TR['personId']==person)&(plans_TR['planElementIndex']==plan_index-1)])[0]
#             persons_baseline.at[i,'vehicle_now'] = vehicle
#             if vehicle2 == 'SF':
#                 persons_baseline.at[i,'project'] = 'CS'
#             if vehicle2 == 'Ca':
#                 persons_baseline.at[i,'project'] = 'CA'
#             if vehicle2 == 'AC':
#                 persons_baseline.at[i,'project'] = '1T'
#         try:

#             persons_baseline.at[i,'vehicle_baseline'] = list(PtoPTss_baseline['vehicleID'][(PtoPTss_baseline['personID']==person)&(PtoPTss_baseline['planIndex']==plan_index)])[0]
#             persons_baseline.at[i,'agency_baseline'] = list(PtoPTss_baseline['vehicle2'][(PtoPTss_baseline['personID']==person)&(PtoPTss_baseline['planIndex']==plan_index)])[0]
#             persons_baseline.at[i,'length_baseline'] = list(PtoPTss_baseline['length'][(PtoPTss_baseline['personID']==person)&(PtoPTss_baseline['planIndex']==plan_index)])[0]
#             persons_baseline.at[i,'length_TR'] = list(PtoPTss_TR['length'][(PtoPTss_baseline['personID']==person)&(PtoPTss_baseline['planIndex']==plan_index)])[0]
#             persons_baseline.at[i,'duration_baseline'] = list(PtoPTss_baseline['duration'][(PtoPTss_baseline['personID']==person)&(PtoPTss_baseline['planIndex']==plan_index)])[0]
#             persons_baseline.at[i,'duration_TR'] = list(PtoPTss_TR['duration'][(PtoPTss_baseline['personID']==person)&(PtoPTss_baseline['planIndex']==plan_index)])[0]
#             persons_baseline.at[i,'speed_baseline'] = persons_baseline.loc[i,'length_baseline']/persons_baseline.loc[i,'duration_baseline']
#             persons_baseline.at[i,'speed_TR'] = persons_baseline.loc[i,'length_TR']/persons_baseline.loc[i,'duration_TR']
#             persons_baseline.at[i,'diff_length'] = persons_baseline.loc[i,'length_TR']-persons_baseline.loc[i,'length_baseline'] 
#             persons_baseline.at[i,'diff_duration'] = persons_baseline.loc[i,'duration_TR']-persons_baseline.loc[i,'duration_baseline'] 
#             persons_baseline.at[i,'diff_speed'] = persons_baseline.loc[i,'speed_TR']-persons_baseline.loc[i,'speed_baseline'] 
#         except:
#             persons_baseline.at[i,'person'] = person
#             persons_baseline.at[i,'plan_index'] = plan_index
#             persons_baseline.at[i,'vehicle_now'] = vehicle
#             if vehicle2 == 'SF':
#                 persons_baseline.at[i,'project'] = 'CS'
#             if vehicle2 == 'Ca':
#                 persons_baseline.at[i,'project'] = 'CA'
#             if vehicle2 == 'AC':
#                 persons_baseline.at[i,'project'] = '1T'

#             print('PtoPTss_baseline not found person and plan index',person,plan_index, vehicle)


# persons_baseline.to_csv('/Users/cpoliziani/Downloads/person_database.csv')


In [ ]:
LIRR_capacities_apr2020 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/apr2020LIRR.csv')
LIRR_capacities_aug2020 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/aug2020LIRR.csv')
LIRR_capacities_aug2021 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/aug2021LIRR.csv')
LIRR_capacities_jan2022 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/jan2022LIRR.csv')
LIRR_capacities_may2022 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/apr2022LIRR.csv')

MNR_capacities_apr2020 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/apr2020MNR.csv')
MNR_capacities_aug2020 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/aug2020MNR.csv')
MNR_capacities_aug2021 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/aug2021MNR.csv')
MNR_capacities_jan2022 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/jan2022MNR.csv')
MNR_capacities_may2022 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/apr2022MNR.csv')

# LIRR_capacities_apr2020 = pd.read_csv('/Users/cpoliziani/Downloads/LIRR_capacities.csv')
# MNR_capacities = MNR_capacities.groupby(['Train']).apply(lambda x: [list(x['Total Seat'])]).to_dict()
# LIRR_capacities = LIRR_capacities.groupby(['Train']).apply(lambda x: [list(x['Total Seats'])]).to_dict()


capacities_data = [LIRR_capacities_apr2020,
        LIRR_capacities_aug2020,
        LIRR_capacities_aug2021,
        LIRR_capacities_jan2022,
        LIRR_capacities_may2022,
        MNR_capacities_apr2020,
        MNR_capacities_aug2020,
        MNR_capacities_aug2021,
        MNR_capacities_jan2022,
        MNR_capacities_may2022,
        ]

for capacity_data in capacities_data:
    trains = []
    for train in capacity_data['Train']:
        try:
            trains.append(int(train))
        except:
            trains.append(train)
    capacity_data['Train'] = trains
    


LIRR_capacities_apr2020 = LIRR_capacities_apr2020.groupby(['Train']).apply(lambda x: [list(x['Total Seats'])]).to_dict()
LIRR_capacities_aug2020 = LIRR_capacities_aug2020.groupby(['Train']).apply(lambda x: [list(x['Total Seats'])]).to_dict()
LIRR_capacities_aug2021 = LIRR_capacities_aug2021.groupby(['Train']).apply(lambda x: [list(x['Total Seats'])]).to_dict()
LIRR_capacities_jan2022 = LIRR_capacities_jan2022.groupby(['Train']).apply(lambda x: [list(x['Total Seats'])]).to_dict()
LIRR_capacities_may2022 = LIRR_capacities_may2022.groupby(['Train']).apply(lambda x: [list(x['Total Seats'])]).to_dict()

MNR_capacities_apr2020 = MNR_capacities_apr2020.groupby(['Train']).apply(lambda x: [list(x['Total Seat'])]).to_dict()
MNR_capacities_aug2020 = MNR_capacities_aug2020.groupby(['Train']).apply(lambda x: [list(x['Total Seat'])]).to_dict()
MNR_capacities_aug2021 = MNR_capacities_aug2021.groupby(['Train']).apply(lambda x: [list(x['Total Seat'])]).to_dict()
MNR_capacities_jan2022 = MNR_capacities_jan2022.groupby(['Train']).apply(lambda x: [list(x['Total Seat'])]).to_dict()
MNR_capacities_may2022 = MNR_capacities_may2022.groupby(['Train']).apply(lambda x: [list(x['Total Seat'])]).to_dict()


LIRR_trips_apr2020 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/r5-prod-1april2020/Long_Island_Rail_20200318/trips.txt')
LIRR_trips_aug2020 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/r5-prod-5august2020/Long_Island_Rail_20200629/trips.txt')
LIRR_trips_aug2021 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/r5-prod-4august2021/Long_Island_Rail_20210726/trips.txt')
LIRR_trips_jan2022 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/r5-prod-5january2022/Long_Island_Rail_20211216/trips.txt')
LIRR_trips_may2022 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/r5-prod-11may2022/Long_Island_Rail_20220430/trips.txt')

LIRR_trips_apr2020['agency_id']='LI'
LIRR_trips_aug2020['agency_id']='LI'
LIRR_trips_aug2021['agency_id']='LI'
LIRR_trips_jan2022['agency_id']='LI'
LIRR_trips_may2022['agency_id']='LI'

MNR_trips_apr2020 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/r5-prod-1april2020/Metro-North_Railroad_20200325/trips.txt')
MNR_trips_aug2020 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/r5-prod-5august2020/Metro-North_Railroad_20200731/trips.txt')
MNR_trips_aug2021 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/r5-prod-4august2021/Metro-North_Railroad_20210721/trips.txt')
MNR_trips_jan2022 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/r5-prod-5january2022/Metro-North_Railroad_20211222/trips.txt')
MNR_trips_may2022 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/r5-prod-11may2022/Metro-North_Railroad_20220429/trips.txt')

MNR_trips_apr2020['agency_id']='1'
MNR_trips_aug2020['agency_id']='1'
MNR_trips_aug2021['agency_id']='1'
MNR_trips_jan2022['agency_id']='1'
MNR_trips_may2022['agency_id']='1'

# MNR_routes_apr2020 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/r5-prod-1april2020/Metro-North_Railroad_20200325/routes.txt')
# MNR_routes_aug2020 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/r5-prod-5august2020/Metro-North_Railroad_20200731/routes.txt')
# MNR_routes_aug2021 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/r5-prod-4august2021/Metro-North_Railroad_20210721/routes.txt')
# MNR_routes_jan2022 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/r5-prod-5january2022/Metro-North_Railroad_20211222/routes.txt')
# MNR_routes_may2022 = pd.read_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/r5-prod-11may2022/Metro-North_Railroad_20220429/routes.txt')

# trips_data_MNR = [MNR_trips_apr2020,
#         MNR_trips_aug2020,
#         MNR_trips_aug2021,
#         MNR_trips_jan2022,
#         MNR_trips_may2022,
#         ]

# routes_data_MNR = [MNR_routes_apr2020,
#         MNR_routes_aug2020,
#         MNR_routes_aug2021,
#         MNR_routes_jan2022,
#         MNR_routes_may2022,
#         ]

# for trip_data_MNR, route_data_MNR, i  in zip(trips_data_MNR,routes_data_MNR, range(len(trips_data))):
#     agencies =[]
#     for route in trip_data_MNR['route_id']:
#         agencies.append(list(route_data_MNR['agency_id'][route_data_MNR['route_id']==route])[0])
#     trip_data_MNR['agency_id']=agencies


# MNR_trips_apr2020 = trips_data_MNR[0]
# MNR_trips_aug2020 = trips_data_MNR[1]
# MNR_trips_aug2021 = trips_data_MNR[2]
# MNR_trips_jan2022 = trips_data_MNR[3]
# MNR_trips_may2022 = trips_data_MNR[4]

trips_data = [LIRR_trips_apr2020,
        LIRR_trips_aug2020,
        LIRR_trips_aug2021,
        LIRR_trips_jan2022,
        LIRR_trips_may2022,
        MNR_trips_apr2020,
        MNR_trips_aug2020,
        MNR_trips_aug2021,
        MNR_trips_jan2022,
        MNR_trips_may2022,
        ]

capacities_data = [LIRR_capacities_apr2020,
        LIRR_capacities_aug2020,
        LIRR_capacities_aug2021,
        LIRR_capacities_jan2022,
        LIRR_capacities_may2022,
        MNR_capacities_apr2020,
        MNR_capacities_aug2020,
        MNR_capacities_aug2021,
        MNR_capacities_jan2022,
        MNR_capacities_may2022,
        ]

for trip_data, capacity_data in zip(trips_data, capacities_data):

    capacities = []
    wrong_ids = []
    print('number of trip', len(trip_data),'number of capacities', len(capacity_data))
    print('number of unique trip', len(np.unique(trip_data['trip_short_name'])))
    for trip_short_name in trip_data['trip_short_name']:
        try:
            capacities.append(capacity_data[trip_short_name][0][0])
        except:
            try:
                capacities.append(capacity_data[float(trip_short_name)][0][0])
            except:
                wrong_ids.append(trip_short_name)
                capacities.append(np.nan)
    trip_data['capacity'] = capacities
    print('Warning!! Not found',len(wrong_ids))
#     print(trip_data)
    print('######################')



In [ ]:
for trip_data in trips_data:
    max_cap = max(list(trip_data['capacity'].dropna()))
    print(max_cap)
    trip_data['capacity'] = trip_data['capacity'].fillna(max_cap)


In [ ]:


trips_data_apr2020 = [trips_data[0],trips_data[5]]
trips_data_aug2020 = [trips_data[1],trips_data[6]]
trips_data_aug2021 = [trips_data[2],trips_data[7]]
trips_data_jan2022 = [trips_data[3],trips_data[8]]
trips_data_may2022 = [trips_data[4],trips_data[9]]

transitVehicleTypesbyTrip = pd.DataFrame(columns = ['agencyId',
                                           'routeId',
                                           'tripId',
                                           'capacity',
                                            'vehicleTypeId',
                                          ])
i=0      
for trip_data, j in zip(trips_data_apr2020,[0,1]):
    for agencyID, tripID, routeID, capacity in zip(trip_data['agency_id'], trip_data['trip_id'],trip_data['route_id'],trip_data['capacity']):
        i+=1
        if i%5000==0:
            print(i)
        transitVehicleTypesbyTrip.at[i,'agencyId'] = agencyID
        transitVehicleTypesbyTrip.at[i,'routeId'] = routeID
        if j == 0:
            transitVehicleTypesbyTrip.at[i,'tripId'] = 'Long_Island_Rail_20200318:'+str(tripID)
        elif j == 1:
            transitVehicleTypesbyTrip.at[i,'tripId'] = 'Metro-North_Railroad_20200325:'+str(tripID)
        transitVehicleTypesbyTrip.at[i,'capacity'] = capacity
        transitVehicleTypesbyTrip.at[i,'vehicleTypeId'] = 'RAIL-DEFAULT'

transitVehicleTypesbyTrip.to_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/transitVehicleTypesbyTrip_apr2020.csv')


transitVehicleTypesbyTrip = pd.DataFrame(columns = ['agencyId',
                                           'routeId',
                                           'tripId',
                                           'capacity',
                                            'vehicleTypeId',
                                          ])
i=0      
for trip_data, j in zip(trips_data_aug2020,[0,1]):
    for agencyID, tripID, routeID, capacity in zip(trip_data['agency_id'], trip_data['trip_id'],trip_data['route_id'],trip_data['capacity']):
        i+=1
        if i%5000==0:
            print(i)
        transitVehicleTypesbyTrip.at[i,'agencyId'] = agencyID
        transitVehicleTypesbyTrip.at[i,'routeId'] = routeID
        if j == 0:
            transitVehicleTypesbyTrip.at[i,'tripId'] = 'Long_Island_Rail_20200629:'+str(tripID)
        elif j == 1:
            transitVehicleTypesbyTrip.at[i,'tripId'] = 'Metro-North_Railroad_20200731:'+str(tripID)
        transitVehicleTypesbyTrip.at[i,'capacity'] = capacity
        transitVehicleTypesbyTrip.at[i,'vehicleTypeId'] = 'RAIL-DEFAULT'

transitVehicleTypesbyTrip.to_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/transitVehicleTypesbyTrip_aug2020.csv')



transitVehicleTypesbyTrip = pd.DataFrame(columns = ['agencyId',
                                           'routeId',
                                           'tripId',
                                           'capacity',
                                            'vehicleTypeId',
                                          ])
i=0      
for trip_data, j in zip(trips_data_aug2021,[0,1]):
    for agencyID, tripID, routeID, capacity in zip(trip_data['agency_id'], trip_data['trip_id'],trip_data['route_id'],trip_data['capacity']):
        i+=1
        if i%5000==0:
            print(i)
        transitVehicleTypesbyTrip.at[i,'agencyId'] = agencyID
        transitVehicleTypesbyTrip.at[i,'routeId'] = routeID
        if j == 0:
            transitVehicleTypesbyTrip.at[i,'tripId'] = 'Long_Island_Rail_20210726:'+str(tripID)
        elif j == 1:
            transitVehicleTypesbyTrip.at[i,'tripId'] = 'Metro-North_Railroad_20210721:'+str(tripID)
        transitVehicleTypesbyTrip.at[i,'capacity'] = capacity
        transitVehicleTypesbyTrip.at[i,'vehicleTypeId'] = 'RAIL-DEFAULT'

transitVehicleTypesbyTrip.to_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/transitVehicleTypesbyTrip_aug2021.csv')



transitVehicleTypesbyTrip = pd.DataFrame(columns = ['agencyId',
                                           'routeId',
                                           'tripId',
                                           'capacity',
                                            'vehicleTypeId',
                                          ])
i=0      
for trip_data, j in zip(trips_data_jan2022,[0,1]):
    for agencyID, tripID, routeID, capacity in zip(trip_data['agency_id'], trip_data['trip_id'],trip_data['route_id'],trip_data['capacity']):
        i+=1
        if i%5000==0:
            print(i)
        transitVehicleTypesbyTrip.at[i,'agencyId'] = agencyID
        transitVehicleTypesbyTrip.at[i,'routeId'] = routeID
        if j == 0:
            transitVehicleTypesbyTrip.at[i,'tripId'] = 'Long_Island_Rail_20211216:'+str(tripID)
        elif j == 1:
            transitVehicleTypesbyTrip.at[i,'tripId'] = 'Metro-North_Railroad_20211222:'+str(tripID)
        transitVehicleTypesbyTrip.at[i,'capacity'] = capacity
        transitVehicleTypesbyTrip.at[i,'vehicleTypeId'] = 'RAIL-DEFAULT'

transitVehicleTypesbyTrip.to_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/transitVehicleTypesbyTrip_jan2022.csv')



transitVehicleTypesbyTrip = pd.DataFrame(columns = ['agencyId',
                                           'routeId',
                                           'tripId',
                                            'capacity',
                                           'vehicleTypeId',
                                          ])
i=0      
for trip_data, j in zip(trips_data_may2022,[0,1]):
    for agencyID, tripID, routeID, capacity in zip(trip_data['agency_id'], trip_data['trip_id'],trip_data['route_id'],trip_data['capacity']):
        i+=1
        if i%5000==0:
            print(i)
        transitVehicleTypesbyTrip.at[i,'agencyId'] = agencyID
        transitVehicleTypesbyTrip.at[i,'routeId'] = routeID
        if j == 0:
            transitVehicleTypesbyTrip.at[i,'tripId'] = 'Long_Island_Rail_20220430:'+str(tripID)
        elif j == 1:
            transitVehicleTypesbyTrip.at[i,'tripId'] = 'Metro-North_Railroad_20220429:'+str(tripID)
        transitVehicleTypesbyTrip.at[i,'capacity'] = capacity
        transitVehicleTypesbyTrip.at[i,'vehicleTypeId'] = 'RAIL-DEFAULT'

transitVehicleTypesbyTrip.to_csv('/Users/cpoliziani/Downloads/Data/EPI/Capacities/transitVehicleTypesbyTrip_may2022.csv')



In [ ]:
trips_data_apr2020[1]

In [ ]:
pd.read_csv('s3://beam-outputs/output/newyork/new-york-jan2022-0-of-10__2022-09-21_17-07-49_qdx/ITERS/it.10/10.events.csv.gz', nrows = 90000)


